In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "3n3ltmfnrqpi",
    "outputId": "a2d4f52a-2fb7-4668-f5f0-fe862aab8eb0",
    "papermill": {
     "duration": 1.489571,
     "end_time": "2020-09-19T06:26:57.168979",
     "exception": false,
     "start_time": "2020-09-19T06:26:55.679408",
     "status": "completed"
    },
    "tags": []
   },
   "source": [
    "<p style=\"text-align:center\">\n",
    "    <a href=\"https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01\" target=\"_blank\">\n",
    "    <img src=\"https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png\" width=\"200\" alt=\"Skills Network Logo\"  />\n",
    "    </a>\n",
    "</p>\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# **Launch Sites Locations Analysis with Folium**\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Estimated time needed: **40** minutes\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Objectives\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "This lab contains the following tasks:\n",
    "\n",
    "*   **TASK 1:** Mark all launch sites on a map\n",
    "*   **TASK 2:** Mark the success/failed launches for each site on the map\n",
    "*   **TASK 3:** Calculate the distances between a launch site to its proximities\n",
    "\n",
    "After completed the above tasks, you should be able to find some geographical patterns about launch sites.\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Let's first import required Python packages for this lab:\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Requirement already satisfied: folium in c:\\programdata\\anaconda3\\lib\\site-packages (0.12.1.post1)\n",
      "Requirement already satisfied: branca>=0.3.0 in c:\\programdata\\anaconda3\\lib\\site-packages (from folium) (0.5.0)\n",
      "Requirement already satisfied: numpy in c:\\programdata\\anaconda3\\lib\\site-packages (from folium) (1.20.3)\n",
      "Requirement already satisfied: requests in c:\\programdata\\anaconda3\\lib\\site-packages (from folium) (2.26.0)\n",
      "Requirement already satisfied: jinja2>=2.9 in c:\\programdata\\anaconda3\\lib\\site-packages (from folium) (2.11.3)\n",
      "Requirement already satisfied: MarkupSafe>=0.23 in c:\\programdata\\anaconda3\\lib\\site-packages (from jinja2>=2.9->folium) (1.1.1)\n",
      "Requirement already satisfied: idna<4,>=2.5 in c:\\programdata\\anaconda3\\lib\\site-packages (from requests->folium) (3.2)\n",
      "Requirement already satisfied: certifi>=2017.4.17 in c:\\programdata\\anaconda3\\lib\\site-packages (from requests->folium) (2021.10.8)\n",
      "Requirement already satisfied: urllib3<1.27,>=1.21.1 in c:\\programdata\\anaconda3\\lib\\site-packages (from requests->folium) (1.26.7)\n",
      "Requirement already satisfied: charset-normalizer~=2.0.0 in c:\\programdata\\anaconda3\\lib\\site-packages (from requests->folium) (2.0.4)\n",
      "Requirement already satisfied: wget in c:\\programdata\\anaconda3\\lib\\site-packages (3.2)\n"
     ]
    }
   ],
   "source": [
    "!pip3 install folium\n",
    "!pip3 install wget"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "import folium\n",
    "import wget\n",
    "import pandas as pd"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import folium MarkerCluster plugin\n",
    "from folium.plugins import MarkerCluster\n",
    "# Import folium MousePosition plugin\n",
    "from folium.plugins import MousePosition\n",
    "# Import folium DivIcon plugin\n",
    "from folium.features import DivIcon"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "If you need to refresh your memory about folium, you may download and refer to this previous folium lab:\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Task 1: Mark all launch sites on a map\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "First, let's try to add each site's location on a map using site's latitude and longitude coordinates\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\r",
      "  0% [                                                                                ]    0 / 7710\r",
      "100% [................................................................................] 7710 / 7710"
     ]
    }
   ],
   "source": [
    "# Download and read the `spacex_launch_geo.csv`\n",
    "spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')\n",
    "spacex_df=pd.read_csv(spacex_csv_file)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now, you can take a look at what are the coordinates for each site.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Launch Site</th>\n",
       "      <th>Lat</th>\n",
       "      <th>Long</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>28.562302</td>\n",
       "      <td>-80.577356</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>28.573255</td>\n",
       "      <td>-80.646895</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "    Launch Site        Lat        Long\n",
       "0   CCAFS LC-40  28.562302  -80.577356\n",
       "1  CCAFS SLC-40  28.563197  -80.576820\n",
       "2    KSC LC-39A  28.573255  -80.646895\n",
       "3   VAFB SLC-4E  34.632834 -120.610745"
      ]
     },
     "execution_count": 5,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`\n",
    "spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]\n",
    "launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()\n",
    "launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]\n",
    "launch_sites_df"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Start location is NASA Johnson Space Center\n",
    "nasa_coordinate = [29.559684888503615, -95.0830971930759]\n",
    "site_map = folium.Map(location=nasa_coordinate, zoom_start=10)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div style=\"width:100%;\"><div style=\"position:relative;width:100%;height:0;padding-bottom:60%;\"><span style=\"color:#565656\">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc=\"&lt;!DOCTYPE html&gt;\n",
       "&lt;head&gt;    \n",
       "    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n",
       "    \n",
       "        &lt;script&gt;\n",
       "            L_NO_TOUCH = false;\n",
       "            L_DISABLE_3D = false;\n",
       "        &lt;/script&gt;\n",
       "    \n",
       "    &lt;style&gt;html, body {width: 100%;height: 100%;margin: 0;padding: 0;}&lt;/style&gt;\n",
       "    &lt;style&gt;#map {position:absolute;top:0;bottom:0;right:0;left:0;}&lt;/style&gt;\n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://code.jquery.com/jquery-1.12.4.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css&quot;/&gt;\n",
       "    \n",
       "            &lt;meta name=&quot;viewport&quot; content=&quot;width=device-width,\n",
       "                initial-scale=1.0, maximum-scale=1.0, user-scalable=no&quot; /&gt;\n",
       "            &lt;style&gt;\n",
       "                #map_2d3a938d8274240a25c97a60abfbee22 {\n",
       "                    position: relative;\n",
       "                    width: 100.0%;\n",
       "                    height: 100.0%;\n",
       "                    left: 0.0%;\n",
       "                    top: 0.0%;\n",
       "                }\n",
       "            &lt;/style&gt;\n",
       "        \n",
       "&lt;/head&gt;\n",
       "&lt;body&gt;    \n",
       "    \n",
       "            &lt;div class=&quot;folium-map&quot; id=&quot;map_2d3a938d8274240a25c97a60abfbee22&quot; &gt;&lt;/div&gt;\n",
       "        \n",
       "&lt;/body&gt;\n",
       "&lt;script&gt;    \n",
       "    \n",
       "            var map_2d3a938d8274240a25c97a60abfbee22 = L.map(\n",
       "                &quot;map_2d3a938d8274240a25c97a60abfbee22&quot;,\n",
       "                {\n",
       "                    center: [29.559684888503615, -95.0830971930759],\n",
       "                    crs: L.CRS.EPSG3857,\n",
       "                    zoom: 10,\n",
       "                    zoomControl: true,\n",
       "                    preferCanvas: false,\n",
       "                }\n",
       "            );\n",
       "\n",
       "            \n",
       "\n",
       "        \n",
       "    \n",
       "            var tile_layer_679011a498912261de28cd751e4c606c = L.tileLayer(\n",
       "                &quot;https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png&quot;,\n",
       "                {&quot;attribution&quot;: &quot;Data by \\u0026copy; \\u003ca href=\\&quot;http://openstreetmap.org\\&quot;\\u003eOpenStreetMap\\u003c/a\\u003e, under \\u003ca href=\\&quot;http://www.openstreetmap.org/copyright\\&quot;\\u003eODbL\\u003c/a\\u003e.&quot;, &quot;detectRetina&quot;: false, &quot;maxNativeZoom&quot;: 18, &quot;maxZoom&quot;: 18, &quot;minZoom&quot;: 0, &quot;noWrap&quot;: false, &quot;opacity&quot;: 1, &quot;subdomains&quot;: &quot;abc&quot;, &quot;tms&quot;: false}\n",
       "            ).addTo(map_2d3a938d8274240a25c97a60abfbee22);\n",
       "        \n",
       "    \n",
       "            var circle_8f26b2f39ccff872a69e48af4a0c4a65 = L.circle(\n",
       "                [29.559684888503615, -95.0830971930759],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 1000, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_2d3a938d8274240a25c97a60abfbee22);\n",
       "        \n",
       "    \n",
       "        var popup_9250d4f1311b6342495e108dd6870244 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_d4b50b5c694967bf7fd688a2dc43e187 = $(`&lt;div id=&quot;html_d4b50b5c694967bf7fd688a2dc43e187&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;NASA Johnson Space Center&lt;/div&gt;`)[0];\n",
       "            popup_9250d4f1311b6342495e108dd6870244.setContent(html_d4b50b5c694967bf7fd688a2dc43e187);\n",
       "        \n",
       "\n",
       "        circle_8f26b2f39ccff872a69e48af4a0c4a65.bindPopup(popup_9250d4f1311b6342495e108dd6870244)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_dd8dd9bf6489ff192d39d12fa90291be = L.marker(\n",
       "                [29.559684888503615, -95.0830971930759],\n",
       "                {}\n",
       "            ).addTo(map_2d3a938d8274240a25c97a60abfbee22);\n",
       "        \n",
       "    \n",
       "            var div_icon_0d808462d5a4ba73a8d12414018e592b = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eNASA JSC\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_dd8dd9bf6489ff192d39d12fa90291be.setIcon(div_icon_0d808462d5a4ba73a8d12414018e592b);\n",
       "        \n",
       "&lt;/script&gt;\" style=\"position:absolute;width:100%;height:100%;left:0;top:0;border:none !important;\" allowfullscreen webkitallowfullscreen mozallowfullscreen></iframe></div></div>"
      ],
      "text/plain": [
       "<folium.folium.Map at 0x228ad61b1c0>"
      ]
     },
     "execution_count": 7,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name\n",
    "circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))\n",
    "# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name\n",
    "marker = folium.map.Marker(\n",
    "    nasa_coordinate,\n",
    "    # Create an icon as a text label\n",
    "    icon=DivIcon(\n",
    "        icon_size=(20,20),\n",
    "        icon_anchor=(0,0),\n",
    "        html='<div style=\"font-size: 12; color:#d35400;\"><b>%s</b></div>' % 'NASA JSC',\n",
    "        )\n",
    "    )\n",
    "site_map.add_child(circle)\n",
    "site_map.add_child(marker)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now, let's add a circle for each launch site in data frame `launch_sites`\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "An example of folium.Circle:\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "An example of folium.Marker:\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style=\"font-size: 12; color:#d35400;\"><b>%s</b></div>' % 'label', ))`\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div style=\"width:100%;\"><div style=\"position:relative;width:100%;height:0;padding-bottom:60%;\"><span style=\"color:#565656\">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc=\"&lt;!DOCTYPE html&gt;\n",
       "&lt;head&gt;    \n",
       "    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n",
       "    \n",
       "        &lt;script&gt;\n",
       "            L_NO_TOUCH = false;\n",
       "            L_DISABLE_3D = false;\n",
       "        &lt;/script&gt;\n",
       "    \n",
       "    &lt;style&gt;html, body {width: 100%;height: 100%;margin: 0;padding: 0;}&lt;/style&gt;\n",
       "    &lt;style&gt;#map {position:absolute;top:0;bottom:0;right:0;left:0;}&lt;/style&gt;\n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://code.jquery.com/jquery-1.12.4.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css&quot;/&gt;\n",
       "    \n",
       "            &lt;meta name=&quot;viewport&quot; content=&quot;width=device-width,\n",
       "                initial-scale=1.0, maximum-scale=1.0, user-scalable=no&quot; /&gt;\n",
       "            &lt;style&gt;\n",
       "                #map_13da6d1993af7ee5c12602e3cf427f4c {\n",
       "                    position: relative;\n",
       "                    width: 100.0%;\n",
       "                    height: 100.0%;\n",
       "                    left: 0.0%;\n",
       "                    top: 0.0%;\n",
       "                }\n",
       "            &lt;/style&gt;\n",
       "        \n",
       "&lt;/head&gt;\n",
       "&lt;body&gt;    \n",
       "    \n",
       "            &lt;div class=&quot;folium-map&quot; id=&quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot; &gt;&lt;/div&gt;\n",
       "        \n",
       "&lt;/body&gt;\n",
       "&lt;script&gt;    \n",
       "    \n",
       "            var map_13da6d1993af7ee5c12602e3cf427f4c = L.map(\n",
       "                &quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot;,\n",
       "                {\n",
       "                    center: [29.559684888503615, -95.0830971930759],\n",
       "                    crs: L.CRS.EPSG3857,\n",
       "                    zoom: 5,\n",
       "                    zoomControl: true,\n",
       "                    preferCanvas: false,\n",
       "                }\n",
       "            );\n",
       "\n",
       "            \n",
       "\n",
       "        \n",
       "    \n",
       "            var tile_layer_2166104b19966fa2881a7383d00f452b = L.tileLayer(\n",
       "                &quot;https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png&quot;,\n",
       "                {&quot;attribution&quot;: &quot;Data by \\u0026copy; \\u003ca href=\\&quot;http://openstreetmap.org\\&quot;\\u003eOpenStreetMap\\u003c/a\\u003e, under \\u003ca href=\\&quot;http://www.openstreetmap.org/copyright\\&quot;\\u003eODbL\\u003c/a\\u003e.&quot;, &quot;detectRetina&quot;: false, &quot;maxNativeZoom&quot;: 18, &quot;maxZoom&quot;: 18, &quot;minZoom&quot;: 0, &quot;noWrap&quot;: false, &quot;opacity&quot;: 1, &quot;subdomains&quot;: &quot;abc&quot;, &quot;tms&quot;: false}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var circle_6f0a849de353a501e2395175a3f761f0 = L.circle(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_7eb844a9bb43ea76d5241a8570f61c80 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_bbf6ec21ca8f656b44111c240ed74632 = $(`&lt;div id=&quot;html_bbf6ec21ca8f656b44111c240ed74632&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS LC-40&lt;/div&gt;`)[0];\n",
       "            popup_7eb844a9bb43ea76d5241a8570f61c80.setContent(html_bbf6ec21ca8f656b44111c240ed74632);\n",
       "        \n",
       "\n",
       "        circle_6f0a849de353a501e2395175a3f761f0.bindPopup(popup_7eb844a9bb43ea76d5241a8570f61c80)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_6e7980b6a2dbef7b07a78d8e75c21208 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_8582033e4e89ebf084fe0a34280e5a10 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS LC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_6e7980b6a2dbef7b07a78d8e75c21208.setIcon(div_icon_8582033e4e89ebf084fe0a34280e5a10);\n",
       "        \n",
       "    \n",
       "            var circle_9ddbd8d227d09fbd16df7f474f9a4965 = L.circle(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_27b3883f7919a1a90bd34cf2dee3d822 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_7a18c57cd9ce75b6155af7fd2f507f6d = $(`&lt;div id=&quot;html_7a18c57cd9ce75b6155af7fd2f507f6d&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS SLC-40&lt;/div&gt;`)[0];\n",
       "            popup_27b3883f7919a1a90bd34cf2dee3d822.setContent(html_7a18c57cd9ce75b6155af7fd2f507f6d);\n",
       "        \n",
       "\n",
       "        circle_9ddbd8d227d09fbd16df7f474f9a4965.bindPopup(popup_27b3883f7919a1a90bd34cf2dee3d822)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_04ad8130bd8c6de62c039df0fd4311e6 = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_652a00034a9bd07f40f830803c7a8298 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS SLC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_04ad8130bd8c6de62c039df0fd4311e6.setIcon(div_icon_652a00034a9bd07f40f830803c7a8298);\n",
       "        \n",
       "    \n",
       "            var circle_9725ee5d1199c9a997aeb208f10a5ef1 = L.circle(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_b9d362124e2a8b78cb4e789073c010bb = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_c27d42fbb1f22f4d0374ed551a2b1239 = $(`&lt;div id=&quot;html_c27d42fbb1f22f4d0374ed551a2b1239&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;KSC LC-39A&lt;/div&gt;`)[0];\n",
       "            popup_b9d362124e2a8b78cb4e789073c010bb.setContent(html_c27d42fbb1f22f4d0374ed551a2b1239);\n",
       "        \n",
       "\n",
       "        circle_9725ee5d1199c9a997aeb208f10a5ef1.bindPopup(popup_b9d362124e2a8b78cb4e789073c010bb)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_494cb0aea92783a942965a83480e1270 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_c5c35e200cae27c8f4dc2254b5a48635 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eKSC LC-39A\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_494cb0aea92783a942965a83480e1270.setIcon(div_icon_c5c35e200cae27c8f4dc2254b5a48635);\n",
       "        \n",
       "    \n",
       "            var circle_cfcec0e8a1aaa030b605e4e24669f0ef = L.circle(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_fe181cc238567a1b0d791b16717c7900 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_cba202f338d926d70f7c2be069646b43 = $(`&lt;div id=&quot;html_cba202f338d926d70f7c2be069646b43&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;VAFB SLC-4E&lt;/div&gt;`)[0];\n",
       "            popup_fe181cc238567a1b0d791b16717c7900.setContent(html_cba202f338d926d70f7c2be069646b43);\n",
       "        \n",
       "\n",
       "        circle_cfcec0e8a1aaa030b605e4e24669f0ef.bindPopup(popup_fe181cc238567a1b0d791b16717c7900)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_8b1736b25197fb6337072b733684a683 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_a3e0cd94354b2b0f7c5690b8129667fc = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eVAFB SLC-4E\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_8b1736b25197fb6337072b733684a683.setIcon(div_icon_a3e0cd94354b2b0f7c5690b8129667fc);\n",
       "        \n",
       "&lt;/script&gt;\" style=\"position:absolute;width:100%;height:100%;left:0;top:0;border:none !important;\" allowfullscreen webkitallowfullscreen mozallowfullscreen></iframe></div></div>"
      ],
      "text/plain": [
       "<folium.folium.Map at 0x228ad5f7070>"
      ]
     },
     "execution_count": 8,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Initial the map\n",
    "site_map = folium.Map(location=nasa_coordinate, zoom_start=5)\n",
    "# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label\n",
    "for index, row in launch_sites_df.iterrows():\n",
    "    coordinate = [row['Lat'],row['Long']]\n",
    "    name = row['Launch Site']\n",
    "    circle = folium.Circle(coordinate, radius=200, color='#d35400', fill=True).add_child(folium.Popup(name))\n",
    "    marker = folium.map.Marker(\n",
    "                coordinate, \n",
    "                icon=DivIcon(\n",
    "                        icon_size=(20,20),\n",
    "                        icon_anchor=(0,0), \n",
    "                        html='<div style=\"font-size: 12; color:#d35400;\"><b>%s</b></div>' % name, ))\n",
    "    site_map.add_child(circle)\n",
    "    site_map.add_child(marker)\n",
    "\n",
    "site_map"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The generated map with marked launch sites should look similar to the following:\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<center>\n",
    "    <img src=\"https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png\" />\n",
    "</center>\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now, you can explore the map by zoom-in/out the marked areas\n",
    ", and try to answer the following questions:\n",
    "\n",
    "*   Are all launch sites in proximity to the Equator line?\n",
    "*   Are all launch sites in very close proximity to the coast?\n",
    "\n",
    "Also please try to explain your findings.\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Answers\n",
    "1. No, the launch sites are ~28-35° from the equator (latitute). That wouldn't count as \"in proximity\"\n",
    "2. Yes, all launch sites are in very close proximity to the coast."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Task 2: Mark the success/failed launches for each site on the map\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.\n",
    "Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Launch Site</th>\n",
       "      <th>Lat</th>\n",
       "      <th>Long</th>\n",
       "      <th>class</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>46</th>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>28.573255</td>\n",
       "      <td>-80.646895</td>\n",
       "      <td>1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>47</th>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>28.573255</td>\n",
       "      <td>-80.646895</td>\n",
       "      <td>1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>48</th>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>28.573255</td>\n",
       "      <td>-80.646895</td>\n",
       "      <td>1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>49</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "      <td>1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>50</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "      <td>1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>51</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>52</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>53</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>54</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "      <td>1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>55</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "      <td>0</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "     Launch Site        Lat       Long  class\n",
       "46    KSC LC-39A  28.573255 -80.646895      1\n",
       "47    KSC LC-39A  28.573255 -80.646895      1\n",
       "48    KSC LC-39A  28.573255 -80.646895      1\n",
       "49  CCAFS SLC-40  28.563197 -80.576820      1\n",
       "50  CCAFS SLC-40  28.563197 -80.576820      1\n",
       "51  CCAFS SLC-40  28.563197 -80.576820      0\n",
       "52  CCAFS SLC-40  28.563197 -80.576820      0\n",
       "53  CCAFS SLC-40  28.563197 -80.576820      0\n",
       "54  CCAFS SLC-40  28.563197 -80.576820      1\n",
       "55  CCAFS SLC-40  28.563197 -80.576820      0"
      ]
     },
     "execution_count": 9,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "spacex_df.tail(10)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Next, let's create markers for all launch records.\n",
    "If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Let's first create a `MarkerCluster` object\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {
    "id": "wP9PVUZ7Jfjt",
    "outputId": "6a3b8164-940c-4b93-9f3d-c655c4a0c683",
    "papermill": {
     "duration": 0.904519,
     "end_time": "2020-09-19T06:27:38.357041",
     "exception": false,
     "start_time": "2020-09-19T06:27:37.452522",
     "status": "completed"
    },
    "tags": []
   },
   "outputs": [],
   "source": [
    "marker_cluster = MarkerCluster()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [],
   "source": [
    "\n",
    "# Apply a function to check the value of `class` column\n",
    "# If class=1, marker_color value will be green\n",
    "# If class=0, marker_color value will be red\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Launch Site</th>\n",
       "      <th>Lat</th>\n",
       "      <th>Long</th>\n",
       "      <th>class</th>\n",
       "      <th>marker_color</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>46</th>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>28.573255</td>\n",
       "      <td>-80.646895</td>\n",
       "      <td>1</td>\n",
       "      <td>green</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>47</th>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>28.573255</td>\n",
       "      <td>-80.646895</td>\n",
       "      <td>1</td>\n",
       "      <td>green</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>48</th>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>28.573255</td>\n",
       "      <td>-80.646895</td>\n",
       "      <td>1</td>\n",
       "      <td>green</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>49</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "      <td>1</td>\n",
       "      <td>green</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>50</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "      <td>1</td>\n",
       "      <td>green</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>51</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>52</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>53</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>54</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "      <td>1</td>\n",
       "      <td>green</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>55</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "     Launch Site        Lat       Long  class marker_color\n",
       "46    KSC LC-39A  28.573255 -80.646895      1        green\n",
       "47    KSC LC-39A  28.573255 -80.646895      1        green\n",
       "48    KSC LC-39A  28.573255 -80.646895      1        green\n",
       "49  CCAFS SLC-40  28.563197 -80.576820      1        green\n",
       "50  CCAFS SLC-40  28.563197 -80.576820      1        green\n",
       "51  CCAFS SLC-40  28.563197 -80.576820      0          red\n",
       "52  CCAFS SLC-40  28.563197 -80.576820      0          red\n",
       "53  CCAFS SLC-40  28.563197 -80.576820      0          red\n",
       "54  CCAFS SLC-40  28.563197 -80.576820      1        green\n",
       "55  CCAFS SLC-40  28.563197 -80.576820      0          red"
      ]
     },
     "execution_count": 12,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Function to assign color to launch outcome\n",
    "def assign_marker_color(launch_outcome):\n",
    "    if launch_outcome == 1:\n",
    "        return 'green'\n",
    "    else:\n",
    "        return 'red'\n",
    "    \n",
    "spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)\n",
    "spacex_df.tail(10)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 50,
   "metadata": {
    "scrolled": true
   },
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div style=\"width:100%;\"><div style=\"position:relative;width:100%;height:0;padding-bottom:60%;\"><span style=\"color:#565656\">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc=\"&lt;!DOCTYPE html&gt;\n",
       "&lt;head&gt;    \n",
       "    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n",
       "    \n",
       "        &lt;script&gt;\n",
       "            L_NO_TOUCH = false;\n",
       "            L_DISABLE_3D = false;\n",
       "        &lt;/script&gt;\n",
       "    \n",
       "    &lt;style&gt;html, body {width: 100%;height: 100%;margin: 0;padding: 0;}&lt;/style&gt;\n",
       "    &lt;style&gt;#map {position:absolute;top:0;bottom:0;right:0;left:0;}&lt;/style&gt;\n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://code.jquery.com/jquery-1.12.4.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css&quot;/&gt;\n",
       "    \n",
       "            &lt;meta name=&quot;viewport&quot; content=&quot;width=device-width,\n",
       "                initial-scale=1.0, maximum-scale=1.0, user-scalable=no&quot; /&gt;\n",
       "            &lt;style&gt;\n",
       "                #map_eaa602c7995a84d5290f1efa9e704bf9 {\n",
       "                    position: relative;\n",
       "                    width: 100.0%;\n",
       "                    height: 100.0%;\n",
       "                    left: 0.0%;\n",
       "                    top: 0.0%;\n",
       "                }\n",
       "            &lt;/style&gt;\n",
       "        \n",
       "    &lt;script src=&quot;https://cdnjs.cloudflare.com/ajax/libs/leaflet.markercluster/1.1.0/leaflet.markercluster.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdnjs.cloudflare.com/ajax/libs/leaflet.markercluster/1.1.0/MarkerCluster.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdnjs.cloudflare.com/ajax/libs/leaflet.markercluster/1.1.0/MarkerCluster.Default.css&quot;/&gt;\n",
       "&lt;/head&gt;\n",
       "&lt;body&gt;    \n",
       "    \n",
       "            &lt;div class=&quot;folium-map&quot; id=&quot;map_eaa602c7995a84d5290f1efa9e704bf9&quot; &gt;&lt;/div&gt;\n",
       "        \n",
       "&lt;/body&gt;\n",
       "&lt;script&gt;    \n",
       "    \n",
       "            var map_eaa602c7995a84d5290f1efa9e704bf9 = L.map(\n",
       "                &quot;map_eaa602c7995a84d5290f1efa9e704bf9&quot;,\n",
       "                {\n",
       "                    center: [29.559684888503615, -95.0830971930759],\n",
       "                    crs: L.CRS.EPSG3857,\n",
       "                    zoom: 5,\n",
       "                    zoomControl: true,\n",
       "                    preferCanvas: false,\n",
       "                }\n",
       "            );\n",
       "\n",
       "            \n",
       "\n",
       "        \n",
       "    \n",
       "            var tile_layer_1c8fda49fad4315e021c37373b6cf938 = L.tileLayer(\n",
       "                &quot;https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png&quot;,\n",
       "                {&quot;attribution&quot;: &quot;Data by \\u0026copy; \\u003ca href=\\&quot;http://openstreetmap.org\\&quot;\\u003eOpenStreetMap\\u003c/a\\u003e, under \\u003ca href=\\&quot;http://www.openstreetmap.org/copyright\\&quot;\\u003eODbL\\u003c/a\\u003e.&quot;, &quot;detectRetina&quot;: false, &quot;maxNativeZoom&quot;: 18, &quot;maxZoom&quot;: 18, &quot;minZoom&quot;: 0, &quot;noWrap&quot;: false, &quot;opacity&quot;: 1, &quot;subdomains&quot;: &quot;abc&quot;, &quot;tms&quot;: false}\n",
       "            ).addTo(map_eaa602c7995a84d5290f1efa9e704bf9);\n",
       "        \n",
       "    \n",
       "            var circle_fa02081d1fb0ac6d579dda8d46408eec = L.circle(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_eaa602c7995a84d5290f1efa9e704bf9);\n",
       "        \n",
       "    \n",
       "        var popup_6fa976524607aaff8e85e7e019c35caf = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_2ba3b784837e4560f9c81c2959b67fcf = $(`&lt;div id=&quot;html_2ba3b784837e4560f9c81c2959b67fcf&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS LC-40&lt;/div&gt;`)[0];\n",
       "            popup_6fa976524607aaff8e85e7e019c35caf.setContent(html_2ba3b784837e4560f9c81c2959b67fcf);\n",
       "        \n",
       "\n",
       "        circle_fa02081d1fb0ac6d579dda8d46408eec.bindPopup(popup_6fa976524607aaff8e85e7e019c35caf)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_59a343ed9466c7f6104a6cf4e73d8afd = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(map_eaa602c7995a84d5290f1efa9e704bf9);\n",
       "        \n",
       "    \n",
       "            var div_icon_5c16bad2c06dd2064e473bd89fcf6682 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS LC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_59a343ed9466c7f6104a6cf4e73d8afd.setIcon(div_icon_5c16bad2c06dd2064e473bd89fcf6682);\n",
       "        \n",
       "    \n",
       "            var circle_6cc00d0dca1ac5225edc7c599457d165 = L.circle(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_eaa602c7995a84d5290f1efa9e704bf9);\n",
       "        \n",
       "    \n",
       "        var popup_2b0fdc59450899862beaf048115dbf9c = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_03e95f37ca73db390b121a6b1b812eb5 = $(`&lt;div id=&quot;html_03e95f37ca73db390b121a6b1b812eb5&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS SLC-40&lt;/div&gt;`)[0];\n",
       "            popup_2b0fdc59450899862beaf048115dbf9c.setContent(html_03e95f37ca73db390b121a6b1b812eb5);\n",
       "        \n",
       "\n",
       "        circle_6cc00d0dca1ac5225edc7c599457d165.bindPopup(popup_2b0fdc59450899862beaf048115dbf9c)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_98a2eeb140bf0386781878e03d1d7e6a = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(map_eaa602c7995a84d5290f1efa9e704bf9);\n",
       "        \n",
       "    \n",
       "            var div_icon_ef7368745b6a053efee1385dc1b84e40 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS SLC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_98a2eeb140bf0386781878e03d1d7e6a.setIcon(div_icon_ef7368745b6a053efee1385dc1b84e40);\n",
       "        \n",
       "    \n",
       "            var circle_50525fdc4bddfa22083d6bb02ed6a64a = L.circle(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_eaa602c7995a84d5290f1efa9e704bf9);\n",
       "        \n",
       "    \n",
       "        var popup_622b6d43b136f7b10ad1030004f528a2 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_9f72ba5a05a2ac69445e7120ab8f5022 = $(`&lt;div id=&quot;html_9f72ba5a05a2ac69445e7120ab8f5022&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;KSC LC-39A&lt;/div&gt;`)[0];\n",
       "            popup_622b6d43b136f7b10ad1030004f528a2.setContent(html_9f72ba5a05a2ac69445e7120ab8f5022);\n",
       "        \n",
       "\n",
       "        circle_50525fdc4bddfa22083d6bb02ed6a64a.bindPopup(popup_622b6d43b136f7b10ad1030004f528a2)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_c45bc0f511bdc5e7a7119aa0840021a2 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(map_eaa602c7995a84d5290f1efa9e704bf9);\n",
       "        \n",
       "    \n",
       "            var div_icon_03a68a9221f9ccb889485edc67c2eabd = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eKSC LC-39A\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_c45bc0f511bdc5e7a7119aa0840021a2.setIcon(div_icon_03a68a9221f9ccb889485edc67c2eabd);\n",
       "        \n",
       "    \n",
       "            var circle_bf8c20ea2a8f8ac7bdb8ec2dc252a16c = L.circle(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_eaa602c7995a84d5290f1efa9e704bf9);\n",
       "        \n",
       "    \n",
       "        var popup_e2ab2b1160d7f0af551fd031cfef8de0 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_b3101cd5f3856895ce1b992bbfcd00e4 = $(`&lt;div id=&quot;html_b3101cd5f3856895ce1b992bbfcd00e4&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;VAFB SLC-4E&lt;/div&gt;`)[0];\n",
       "            popup_e2ab2b1160d7f0af551fd031cfef8de0.setContent(html_b3101cd5f3856895ce1b992bbfcd00e4);\n",
       "        \n",
       "\n",
       "        circle_bf8c20ea2a8f8ac7bdb8ec2dc252a16c.bindPopup(popup_e2ab2b1160d7f0af551fd031cfef8de0)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_a0e6da365e5ab43befe9b7c6903b0b69 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(map_eaa602c7995a84d5290f1efa9e704bf9);\n",
       "        \n",
       "    \n",
       "            var div_icon_e9b014c463d1f83c55b63b887f41e17d = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eVAFB SLC-4E\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_a0e6da365e5ab43befe9b7c6903b0b69.setIcon(div_icon_e9b014c463d1f83c55b63b887f41e17d);\n",
       "        \n",
       "    \n",
       "            var marker_cluster_4577326dc3c19326c96ee964ab36dadf = L.markerClusterGroup(\n",
       "                {}\n",
       "            );\n",
       "            map_eaa602c7995a84d5290f1efa9e704bf9.addLayer(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var marker_cc26367b8a7fa0c3bf97d8aaf59796fa = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_83e0a4762a7c97105a7c91c068144594 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_cc26367b8a7fa0c3bf97d8aaf59796fa.setIcon(icon_83e0a4762a7c97105a7c91c068144594);\n",
       "        \n",
       "    \n",
       "            var marker_b45d25082211b2776cbb0dd788cfa321 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_0810330d5b3bdc6045cbeadba8c011f2 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_b45d25082211b2776cbb0dd788cfa321.setIcon(icon_0810330d5b3bdc6045cbeadba8c011f2);\n",
       "        \n",
       "    \n",
       "            var marker_a8c2a6a7ba79383496da9603b2642f89 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_a5f7b151273b92a718443ed32e635448 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_a8c2a6a7ba79383496da9603b2642f89.setIcon(icon_a5f7b151273b92a718443ed32e635448);\n",
       "        \n",
       "    \n",
       "            var marker_4b9c0b80c46501858a834c3a9cf3a78b = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_d471f9827406ed8527881732d74805e6 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_4b9c0b80c46501858a834c3a9cf3a78b.setIcon(icon_d471f9827406ed8527881732d74805e6);\n",
       "        \n",
       "    \n",
       "            var marker_617e09d6b934543a6b25a73771435fb3 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_92846db88c332422899ecb0460055523 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_617e09d6b934543a6b25a73771435fb3.setIcon(icon_92846db88c332422899ecb0460055523);\n",
       "        \n",
       "    \n",
       "            var marker_0492c4e8799795136fb97b99dca275bc = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_aa0a53a3d57a3e7e89c111936fab15b1 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_0492c4e8799795136fb97b99dca275bc.setIcon(icon_aa0a53a3d57a3e7e89c111936fab15b1);\n",
       "        \n",
       "    \n",
       "            var marker_e7acbc2551381289617ce09a7fba6f8f = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_9b92ae313766d0f2d75ecc1a2ec4d41f = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_e7acbc2551381289617ce09a7fba6f8f.setIcon(icon_9b92ae313766d0f2d75ecc1a2ec4d41f);\n",
       "        \n",
       "    \n",
       "            var marker_f4c45b60f129eb6f8a3dbec71ea8d4af = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_1e35c5b2c7e1ee663ff8ae8348e165f9 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_f4c45b60f129eb6f8a3dbec71ea8d4af.setIcon(icon_1e35c5b2c7e1ee663ff8ae8348e165f9);\n",
       "        \n",
       "    \n",
       "            var marker_b17aa3661893a15ffc12dd8b5fa0d450 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_6c7d6b63cc1d2ece017324f8511fc151 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_b17aa3661893a15ffc12dd8b5fa0d450.setIcon(icon_6c7d6b63cc1d2ece017324f8511fc151);\n",
       "        \n",
       "    \n",
       "            var marker_a788a7921ea312cec5765f7d1b38545e = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_835df652c6bfa0486574fa58465ae054 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_a788a7921ea312cec5765f7d1b38545e.setIcon(icon_835df652c6bfa0486574fa58465ae054);\n",
       "        \n",
       "    \n",
       "            var marker_1f3274db44e4a96f8f503fa3d174241a = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_3a8cdf40e45098da0b89b2ca9ba03d4e = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_1f3274db44e4a96f8f503fa3d174241a.setIcon(icon_3a8cdf40e45098da0b89b2ca9ba03d4e);\n",
       "        \n",
       "    \n",
       "            var marker_a3afe74c55cfebf87169d81945ae85c8 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_26e0eee95ffd11f259f6d88fb8b71f99 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_a3afe74c55cfebf87169d81945ae85c8.setIcon(icon_26e0eee95ffd11f259f6d88fb8b71f99);\n",
       "        \n",
       "    \n",
       "            var marker_5306770926beda68ee7d06f562000fad = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_eacc179087016679c2f1d82acf91fe7e = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_5306770926beda68ee7d06f562000fad.setIcon(icon_eacc179087016679c2f1d82acf91fe7e);\n",
       "        \n",
       "    \n",
       "            var marker_5159ba3812dcec27111f2af3f43294f6 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_7c5ad6491200a5f140725994d79dd115 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_5159ba3812dcec27111f2af3f43294f6.setIcon(icon_7c5ad6491200a5f140725994d79dd115);\n",
       "        \n",
       "    \n",
       "            var marker_eae9411bc59a8a477452bffcc3e21fef = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_c229cfb20a1e1849638f80a6ecf08b01 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_eae9411bc59a8a477452bffcc3e21fef.setIcon(icon_c229cfb20a1e1849638f80a6ecf08b01);\n",
       "        \n",
       "    \n",
       "            var marker_330e0cbd7232d0063cea3f2bc261d95d = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_81cd9ff3c21f2eccbb60351b28dc291c = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_330e0cbd7232d0063cea3f2bc261d95d.setIcon(icon_81cd9ff3c21f2eccbb60351b28dc291c);\n",
       "        \n",
       "    \n",
       "            var marker_d66749a40bdc65b5d09ac5dd7a901953 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_3fed0838a03fe4520c6b7ac5d5480963 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_d66749a40bdc65b5d09ac5dd7a901953.setIcon(icon_3fed0838a03fe4520c6b7ac5d5480963);\n",
       "        \n",
       "    \n",
       "            var marker_92d4a019c39b68db57cce7c7c2fc00c7 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_ed1af63ea60481e871363a6119c476b3 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_92d4a019c39b68db57cce7c7c2fc00c7.setIcon(icon_ed1af63ea60481e871363a6119c476b3);\n",
       "        \n",
       "    \n",
       "            var marker_a94cfe622923cde9740d7ca6d0b2b377 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_bed45e1a9d32399452ea2e43426e8c5c = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_a94cfe622923cde9740d7ca6d0b2b377.setIcon(icon_bed45e1a9d32399452ea2e43426e8c5c);\n",
       "        \n",
       "    \n",
       "            var marker_5b56b8df99b72abb7f1e1fbcaa81765e = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_05f3743235fb31c5d72ebd93b52502e8 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_5b56b8df99b72abb7f1e1fbcaa81765e.setIcon(icon_05f3743235fb31c5d72ebd93b52502e8);\n",
       "        \n",
       "    \n",
       "            var marker_dd403dacfc5a23f3b618a5d1a5fa5b37 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_9bc296a77c756944053b713a75097aa2 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_dd403dacfc5a23f3b618a5d1a5fa5b37.setIcon(icon_9bc296a77c756944053b713a75097aa2);\n",
       "        \n",
       "    \n",
       "            var marker_030caec93d3cbf3e95f6f60f6a0c4d88 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_bf95226e3a1205b935d90bb9f99456f4 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_030caec93d3cbf3e95f6f60f6a0c4d88.setIcon(icon_bf95226e3a1205b935d90bb9f99456f4);\n",
       "        \n",
       "    \n",
       "            var marker_a0193a8b64f273043879dbb420cada4c = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_d0c39fa187827583acd8aa036cd61acc = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_a0193a8b64f273043879dbb420cada4c.setIcon(icon_d0c39fa187827583acd8aa036cd61acc);\n",
       "        \n",
       "    \n",
       "            var marker_6e57c855d2646655e7a2a4855eb2e00a = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_2f811b0f48a9c8ddad02966bd7d74da3 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_6e57c855d2646655e7a2a4855eb2e00a.setIcon(icon_2f811b0f48a9c8ddad02966bd7d74da3);\n",
       "        \n",
       "    \n",
       "            var marker_600c43bb7965e81aa17369828694cc42 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_74d4add40496b17e78edab6f5ab88ffe = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_600c43bb7965e81aa17369828694cc42.setIcon(icon_74d4add40496b17e78edab6f5ab88ffe);\n",
       "        \n",
       "    \n",
       "            var marker_78ca6246d5bc9726f19bbe8fdbc5848b = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_3d190fb4d61973ebee73236c355bae59 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_78ca6246d5bc9726f19bbe8fdbc5848b.setIcon(icon_3d190fb4d61973ebee73236c355bae59);\n",
       "        \n",
       "    \n",
       "            var marker_f2789086163f1441b2088f2b446ce152 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_fddbc7766b155b6b3ba25b79a3e89cf7 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_f2789086163f1441b2088f2b446ce152.setIcon(icon_fddbc7766b155b6b3ba25b79a3e89cf7);\n",
       "        \n",
       "    \n",
       "            var marker_40e14c21d47df0ef038778d1774c0877 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_849aaa1055a2cd3d681be664e8ff49cd = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_40e14c21d47df0ef038778d1774c0877.setIcon(icon_849aaa1055a2cd3d681be664e8ff49cd);\n",
       "        \n",
       "    \n",
       "            var marker_afbda9715834d7f7bb0050e99fd3a6f1 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_034cb6b58e530dbe6f3c3dfa2b9a85e5 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_afbda9715834d7f7bb0050e99fd3a6f1.setIcon(icon_034cb6b58e530dbe6f3c3dfa2b9a85e5);\n",
       "        \n",
       "    \n",
       "            var marker_0735a50c41464704bd4938cbcc44a188 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_c1588f3fb89dd5832c9752c4caf48b97 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_0735a50c41464704bd4938cbcc44a188.setIcon(icon_c1588f3fb89dd5832c9752c4caf48b97);\n",
       "        \n",
       "    \n",
       "            var marker_15b7d3357c91941177853a7f9dd9ca74 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_c03dc3447e1de4008fefe594bea62d87 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_15b7d3357c91941177853a7f9dd9ca74.setIcon(icon_c03dc3447e1de4008fefe594bea62d87);\n",
       "        \n",
       "    \n",
       "            var marker_98c7d83b3863124e9594827a4a49ab6d = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_c21fcabf429f0a388b06f6eddc00a535 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_98c7d83b3863124e9594827a4a49ab6d.setIcon(icon_c21fcabf429f0a388b06f6eddc00a535);\n",
       "        \n",
       "    \n",
       "            var marker_d0d43b585a2a7a1b79cc2c532ed69bf8 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_86f9a65736a42798514d8372266d8a24 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_d0d43b585a2a7a1b79cc2c532ed69bf8.setIcon(icon_86f9a65736a42798514d8372266d8a24);\n",
       "        \n",
       "    \n",
       "            var marker_c816600c8ba137c0cb7394eef2809436 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_0fd12fd4c282053f5055c4bdf101cd54 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_c816600c8ba137c0cb7394eef2809436.setIcon(icon_0fd12fd4c282053f5055c4bdf101cd54);\n",
       "        \n",
       "    \n",
       "            var marker_485dbca367551ea9bc9d800048267931 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_a98ead1b93c9daf66d404f7621b357a4 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_485dbca367551ea9bc9d800048267931.setIcon(icon_a98ead1b93c9daf66d404f7621b357a4);\n",
       "        \n",
       "    \n",
       "            var marker_3f1199e417dec4b167c9e51cdcc38738 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_e7e4f746c356a1d5536159aa79901226 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_3f1199e417dec4b167c9e51cdcc38738.setIcon(icon_e7e4f746c356a1d5536159aa79901226);\n",
       "        \n",
       "    \n",
       "            var marker_344070c2cc9ea9442f05f8aa435eacd1 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_2dec13bc833b04aa1f487946728ff62c = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_344070c2cc9ea9442f05f8aa435eacd1.setIcon(icon_2dec13bc833b04aa1f487946728ff62c);\n",
       "        \n",
       "    \n",
       "            var marker_1a1f2a94581ccaced3cb860c4d0f8ec4 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_0008b69ee2e7ffb4599eb9fdeec8f2e3 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_1a1f2a94581ccaced3cb860c4d0f8ec4.setIcon(icon_0008b69ee2e7ffb4599eb9fdeec8f2e3);\n",
       "        \n",
       "    \n",
       "            var marker_293c0a87597affffb431e181dff56986 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_4ee34b665b916e223c2b3d862abf4885 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_293c0a87597affffb431e181dff56986.setIcon(icon_4ee34b665b916e223c2b3d862abf4885);\n",
       "        \n",
       "    \n",
       "            var marker_9d8cca6e0a502f5b326058bb3d13fbd7 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_422a63a3e7205a6365208827a5912dbd = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_9d8cca6e0a502f5b326058bb3d13fbd7.setIcon(icon_422a63a3e7205a6365208827a5912dbd);\n",
       "        \n",
       "    \n",
       "            var marker_4d24197598576e34e09a01a7cafd1e30 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_78c4af4a1d3cab76f21f27630ca94396 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_4d24197598576e34e09a01a7cafd1e30.setIcon(icon_78c4af4a1d3cab76f21f27630ca94396);\n",
       "        \n",
       "    \n",
       "            var marker_f5b191942d8a14a577de1e5cc2cff429 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_2993b4e2a193a4d1255b7af59dfc4cf4 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_f5b191942d8a14a577de1e5cc2cff429.setIcon(icon_2993b4e2a193a4d1255b7af59dfc4cf4);\n",
       "        \n",
       "    \n",
       "            var marker_0a37eb562ffae83b89a5a8736c1a7945 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_101b60307330c5febba63ca31d3b260b = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_0a37eb562ffae83b89a5a8736c1a7945.setIcon(icon_101b60307330c5febba63ca31d3b260b);\n",
       "        \n",
       "    \n",
       "            var marker_0a6549ac241c2ef20a1179178c4bad70 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_a49e151aa527f8f093b87d756f378499 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_0a6549ac241c2ef20a1179178c4bad70.setIcon(icon_a49e151aa527f8f093b87d756f378499);\n",
       "        \n",
       "    \n",
       "            var marker_fe9cd7fb4bbc4b36764af4f18bebcd69 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_7b77429e9815ab5494fe561342f8e7d2 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_fe9cd7fb4bbc4b36764af4f18bebcd69.setIcon(icon_7b77429e9815ab5494fe561342f8e7d2);\n",
       "        \n",
       "    \n",
       "            var marker_593b88f08ea65783556822ef5d090109 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_812f2cc23ab04b4bd2b6c6df4c2c07c3 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_593b88f08ea65783556822ef5d090109.setIcon(icon_812f2cc23ab04b4bd2b6c6df4c2c07c3);\n",
       "        \n",
       "    \n",
       "            var marker_e275a4e702727b71c51c1dc531b106b2 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_0e4c00922c6c5636c0cb51ab939280c8 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_e275a4e702727b71c51c1dc531b106b2.setIcon(icon_0e4c00922c6c5636c0cb51ab939280c8);\n",
       "        \n",
       "    \n",
       "            var marker_bd5bd08fc8acef1e082719968dc70079 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_5ebf7e9342778647c9c14ac636d47853 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_bd5bd08fc8acef1e082719968dc70079.setIcon(icon_5ebf7e9342778647c9c14ac636d47853);\n",
       "        \n",
       "    \n",
       "            var marker_5576ac34e228a0656a7f3aa0aec6b6a9 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_3bb18fd2134775ab8e8e0066d63a175c = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_5576ac34e228a0656a7f3aa0aec6b6a9.setIcon(icon_3bb18fd2134775ab8e8e0066d63a175c);\n",
       "        \n",
       "    \n",
       "            var marker_ffa7da8e0a4bde196c150fdcfbebc574 = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_c32760d15f3244dbfb03bb3b0413a904 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_ffa7da8e0a4bde196c150fdcfbebc574.setIcon(icon_c32760d15f3244dbfb03bb3b0413a904);\n",
       "        \n",
       "    \n",
       "            var marker_081d740086416ee0de0c65490e462fbb = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_56037f26e483095d988b8e15bb4b731f = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_081d740086416ee0de0c65490e462fbb.setIcon(icon_56037f26e483095d988b8e15bb4b731f);\n",
       "        \n",
       "    \n",
       "            var marker_e60358edfcea965bb34fa8e03ef06ccf = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_6818b6af184a7a58165369b9ede3e90c = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_e60358edfcea965bb34fa8e03ef06ccf.setIcon(icon_6818b6af184a7a58165369b9ede3e90c);\n",
       "        \n",
       "    \n",
       "            var marker_d80181abdb548ba1d379ce618ce26d71 = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_e76c985c34c1b4147963f9457d8fb2ae = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_d80181abdb548ba1d379ce618ce26d71.setIcon(icon_e76c985c34c1b4147963f9457d8fb2ae);\n",
       "        \n",
       "    \n",
       "            var marker_b5c7e92bbd89cc1ece77202b0532b4e7 = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_b4683f857675ffd1ceb4357600739475 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_b5c7e92bbd89cc1ece77202b0532b4e7.setIcon(icon_b4683f857675ffd1ceb4357600739475);\n",
       "        \n",
       "    \n",
       "            var marker_c896ef78385ce5823db537a07c90cea3 = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_da818429210e97ddab41381ff1242154 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;green&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_c896ef78385ce5823db537a07c90cea3.setIcon(icon_da818429210e97ddab41381ff1242154);\n",
       "        \n",
       "    \n",
       "            var marker_de6430d4922e9fa2be3e2e6119c1f032 = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(marker_cluster_4577326dc3c19326c96ee964ab36dadf);\n",
       "        \n",
       "    \n",
       "            var icon_838399ec911a6bc76a4cf48737d71494 = L.AwesomeMarkers.icon(\n",
       "                {&quot;extraClasses&quot;: &quot;fa-rotate-0&quot;, &quot;icon&quot;: &quot;info-sign&quot;, &quot;iconColor&quot;: &quot;red&quot;, &quot;markerColor&quot;: &quot;white&quot;, &quot;prefix&quot;: &quot;glyphicon&quot;}\n",
       "            );\n",
       "            marker_de6430d4922e9fa2be3e2e6119c1f032.setIcon(icon_838399ec911a6bc76a4cf48737d71494);\n",
       "        \n",
       "&lt;/script&gt;\" style=\"position:absolute;width:100%;height:100%;left:0;top:0;border:none !important;\" allowfullscreen webkitallowfullscreen mozallowfullscreen></iframe></div></div>"
      ],
      "text/plain": [
       "<folium.folium.Map at 0x1fb6ad4aaf0>"
      ]
     },
     "execution_count": 50,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Add marker_cluster to current site_map\n",
    "site_map.add_child(marker_cluster)\n",
    "\n",
    "# for each row in spacex_df data frame\n",
    "# create a Marker object with its coordinate\n",
    "# and customize the Marker's icon property to indicate if this launch was successed or failed, \n",
    "# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']\n",
    "for index, record in spacex_df.iterrows():\n",
    "    # TODO: Create and add a Marker cluster to the site map\n",
    "    marker = folium.Marker(\n",
    "        location = [record['Lat'], record['Long']],\n",
    "        icon = folium.Icon(color='white', icon_color=record['marker_color'])\n",
    "    )\n",
    "    marker_cluster.add_child(marker)\n",
    "\n",
    "site_map"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Your updated map may look like the following screenshots:\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<center>\n",
    "    <img src=\"https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png\" />\n",
    "</center>\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<center>\n",
    "    <img src=\"https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png\" />\n",
    "</center>\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# TASK 3: Calculate the distances between a launch site to its proximities\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Next, we need to explore and analyze the proximities of launch sites.\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div style=\"width:100%;\"><div style=\"position:relative;width:100%;height:0;padding-bottom:60%;\"><span style=\"color:#565656\">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc=\"&lt;!DOCTYPE html&gt;\n",
       "&lt;head&gt;    \n",
       "    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n",
       "    \n",
       "        &lt;script&gt;\n",
       "            L_NO_TOUCH = false;\n",
       "            L_DISABLE_3D = false;\n",
       "        &lt;/script&gt;\n",
       "    \n",
       "    &lt;style&gt;html, body {width: 100%;height: 100%;margin: 0;padding: 0;}&lt;/style&gt;\n",
       "    &lt;style&gt;#map {position:absolute;top:0;bottom:0;right:0;left:0;}&lt;/style&gt;\n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://code.jquery.com/jquery-1.12.4.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css&quot;/&gt;\n",
       "    \n",
       "            &lt;meta name=&quot;viewport&quot; content=&quot;width=device-width,\n",
       "                initial-scale=1.0, maximum-scale=1.0, user-scalable=no&quot; /&gt;\n",
       "            &lt;style&gt;\n",
       "                #map_13da6d1993af7ee5c12602e3cf427f4c {\n",
       "                    position: relative;\n",
       "                    width: 100.0%;\n",
       "                    height: 100.0%;\n",
       "                    left: 0.0%;\n",
       "                    top: 0.0%;\n",
       "                }\n",
       "            &lt;/style&gt;\n",
       "        \n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/gh/ardhi/Leaflet.MousePosition/src/L.Control.MousePosition.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/ardhi/Leaflet.MousePosition/src/L.Control.MousePosition.min.css&quot;/&gt;\n",
       "&lt;/head&gt;\n",
       "&lt;body&gt;    \n",
       "    \n",
       "            &lt;div class=&quot;folium-map&quot; id=&quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot; &gt;&lt;/div&gt;\n",
       "        \n",
       "&lt;/body&gt;\n",
       "&lt;script&gt;    \n",
       "    \n",
       "            var map_13da6d1993af7ee5c12602e3cf427f4c = L.map(\n",
       "                &quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot;,\n",
       "                {\n",
       "                    center: [29.559684888503615, -95.0830971930759],\n",
       "                    crs: L.CRS.EPSG3857,\n",
       "                    zoom: 5,\n",
       "                    zoomControl: true,\n",
       "                    preferCanvas: false,\n",
       "                }\n",
       "            );\n",
       "\n",
       "            \n",
       "\n",
       "        \n",
       "    \n",
       "            var tile_layer_2166104b19966fa2881a7383d00f452b = L.tileLayer(\n",
       "                &quot;https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png&quot;,\n",
       "                {&quot;attribution&quot;: &quot;Data by \\u0026copy; \\u003ca href=\\&quot;http://openstreetmap.org\\&quot;\\u003eOpenStreetMap\\u003c/a\\u003e, under \\u003ca href=\\&quot;http://www.openstreetmap.org/copyright\\&quot;\\u003eODbL\\u003c/a\\u003e.&quot;, &quot;detectRetina&quot;: false, &quot;maxNativeZoom&quot;: 18, &quot;maxZoom&quot;: 18, &quot;minZoom&quot;: 0, &quot;noWrap&quot;: false, &quot;opacity&quot;: 1, &quot;subdomains&quot;: &quot;abc&quot;, &quot;tms&quot;: false}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var circle_6f0a849de353a501e2395175a3f761f0 = L.circle(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_7eb844a9bb43ea76d5241a8570f61c80 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_bbf6ec21ca8f656b44111c240ed74632 = $(`&lt;div id=&quot;html_bbf6ec21ca8f656b44111c240ed74632&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS LC-40&lt;/div&gt;`)[0];\n",
       "            popup_7eb844a9bb43ea76d5241a8570f61c80.setContent(html_bbf6ec21ca8f656b44111c240ed74632);\n",
       "        \n",
       "\n",
       "        circle_6f0a849de353a501e2395175a3f761f0.bindPopup(popup_7eb844a9bb43ea76d5241a8570f61c80)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_6e7980b6a2dbef7b07a78d8e75c21208 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_8582033e4e89ebf084fe0a34280e5a10 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS LC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_6e7980b6a2dbef7b07a78d8e75c21208.setIcon(div_icon_8582033e4e89ebf084fe0a34280e5a10);\n",
       "        \n",
       "    \n",
       "            var circle_9ddbd8d227d09fbd16df7f474f9a4965 = L.circle(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_27b3883f7919a1a90bd34cf2dee3d822 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_7a18c57cd9ce75b6155af7fd2f507f6d = $(`&lt;div id=&quot;html_7a18c57cd9ce75b6155af7fd2f507f6d&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS SLC-40&lt;/div&gt;`)[0];\n",
       "            popup_27b3883f7919a1a90bd34cf2dee3d822.setContent(html_7a18c57cd9ce75b6155af7fd2f507f6d);\n",
       "        \n",
       "\n",
       "        circle_9ddbd8d227d09fbd16df7f474f9a4965.bindPopup(popup_27b3883f7919a1a90bd34cf2dee3d822)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_04ad8130bd8c6de62c039df0fd4311e6 = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_652a00034a9bd07f40f830803c7a8298 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS SLC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_04ad8130bd8c6de62c039df0fd4311e6.setIcon(div_icon_652a00034a9bd07f40f830803c7a8298);\n",
       "        \n",
       "    \n",
       "            var circle_9725ee5d1199c9a997aeb208f10a5ef1 = L.circle(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_b9d362124e2a8b78cb4e789073c010bb = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_c27d42fbb1f22f4d0374ed551a2b1239 = $(`&lt;div id=&quot;html_c27d42fbb1f22f4d0374ed551a2b1239&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;KSC LC-39A&lt;/div&gt;`)[0];\n",
       "            popup_b9d362124e2a8b78cb4e789073c010bb.setContent(html_c27d42fbb1f22f4d0374ed551a2b1239);\n",
       "        \n",
       "\n",
       "        circle_9725ee5d1199c9a997aeb208f10a5ef1.bindPopup(popup_b9d362124e2a8b78cb4e789073c010bb)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_494cb0aea92783a942965a83480e1270 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_c5c35e200cae27c8f4dc2254b5a48635 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eKSC LC-39A\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_494cb0aea92783a942965a83480e1270.setIcon(div_icon_c5c35e200cae27c8f4dc2254b5a48635);\n",
       "        \n",
       "    \n",
       "            var circle_cfcec0e8a1aaa030b605e4e24669f0ef = L.circle(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_fe181cc238567a1b0d791b16717c7900 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_cba202f338d926d70f7c2be069646b43 = $(`&lt;div id=&quot;html_cba202f338d926d70f7c2be069646b43&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;VAFB SLC-4E&lt;/div&gt;`)[0];\n",
       "            popup_fe181cc238567a1b0d791b16717c7900.setContent(html_cba202f338d926d70f7c2be069646b43);\n",
       "        \n",
       "\n",
       "        circle_cfcec0e8a1aaa030b605e4e24669f0ef.bindPopup(popup_fe181cc238567a1b0d791b16717c7900)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_8b1736b25197fb6337072b733684a683 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_a3e0cd94354b2b0f7c5690b8129667fc = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eVAFB SLC-4E\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_8b1736b25197fb6337072b733684a683.setIcon(div_icon_a3e0cd94354b2b0f7c5690b8129667fc);\n",
       "        \n",
       "    \n",
       "            var mouse_position_12f39c56fd7cd89bffa34218c48112aa = new L.Control.MousePosition(\n",
       "                {&quot;emptyString&quot;: &quot;NaN&quot;, &quot;lngFirst&quot;: false, &quot;numDigits&quot;: 20, &quot;position&quot;: &quot;topright&quot;, &quot;prefix&quot;: &quot;Lat:&quot;, &quot;separator&quot;: &quot; Long: &quot;}\n",
       "            );\n",
       "            mouse_position_12f39c56fd7cd89bffa34218c48112aa.options[&quot;latFormatter&quot;] =\n",
       "                function(num) {return L.Util.formatNum(num, 5);};;\n",
       "            mouse_position_12f39c56fd7cd89bffa34218c48112aa.options[&quot;lngFormatter&quot;] =\n",
       "                function(num) {return L.Util.formatNum(num, 5);};;\n",
       "            map_13da6d1993af7ee5c12602e3cf427f4c.addControl(mouse_position_12f39c56fd7cd89bffa34218c48112aa);\n",
       "        \n",
       "&lt;/script&gt;\" style=\"position:absolute;width:100%;height:100%;left:0;top:0;border:none !important;\" allowfullscreen webkitallowfullscreen mozallowfullscreen></iframe></div></div>"
      ],
      "text/plain": [
       "<folium.folium.Map at 0x228ad5f7070>"
      ]
     },
     "execution_count": 13,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map\n",
    "formatter = \"function(num) {return L.Util.formatNum(num, 5);};\"\n",
    "mouse_position = MousePosition(\n",
    "    position='topright',\n",
    "    separator=' Long: ',\n",
    "    empty_string='NaN',\n",
    "    lng_first=False,\n",
    "    num_digits=20,\n",
    "    prefix='Lat:',\n",
    "    lat_formatter=formatter,\n",
    "    lng_formatter=formatter,\n",
    ")\n",
    "\n",
    "site_map.add_child(mouse_position)\n",
    "site_map"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {},
   "outputs": [],
   "source": [
    "from math import sin, cos, sqrt, atan2, radians\n",
    "\n",
    "def calculate_distance(lat1, lon1, lat2, lon2):\n",
    "    # approximate radius of earth in km\n",
    "    R = 6373.0\n",
    "\n",
    "    lat1 = radians(lat1)\n",
    "    lon1 = radians(lon1)\n",
    "    lat2 = radians(lat2)\n",
    "    lon2 = radians(lon2)\n",
    "\n",
    "    dlon = lon2 - lon1\n",
    "    dlat = lat2 - lat1\n",
    "\n",
    "    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2\n",
    "    c = 2 * atan2(sqrt(a), sqrt(1 - a))\n",
    "\n",
    "    distance = R * c\n",
    "    return distance"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Launch Site</th>\n",
       "      <th>Lat</th>\n",
       "      <th>Long</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>28.562302</td>\n",
       "      <td>-80.577356</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>28.563197</td>\n",
       "      <td>-80.576820</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>28.573255</td>\n",
       "      <td>-80.646895</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "    Launch Site        Lat        Long\n",
       "0   CCAFS LC-40  28.562302  -80.577356\n",
       "1  CCAFS SLC-40  28.563197  -80.576820\n",
       "2    KSC LC-39A  28.573255  -80.646895\n",
       "3   VAFB SLC-4E  34.632834 -120.610745"
      ]
     },
     "execution_count": 15,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "launch_sites_df"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "0.8578845652375999\n"
     ]
    }
   ],
   "source": [
    "# find coordinate of the closet coastline\n",
    "# e.g.,: Lat: 28.56367  Lon: -80.57163\n",
    "# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)\n",
    "# CCAFS SLC 40:\n",
    "coastline_coord = [28.56385, -80.56807]\n",
    "distance_coastline = calculate_distance(\n",
    "    launch_sites_df.loc[1, 'Lat'], launch_sites_df.loc[1, 'Long'], \n",
    "    coastline_coord[0], coastline_coord[1])\n",
    "print(distance_coastline)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div style=\"width:100%;\"><div style=\"position:relative;width:100%;height:0;padding-bottom:60%;\"><span style=\"color:#565656\">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc=\"&lt;!DOCTYPE html&gt;\n",
       "&lt;head&gt;    \n",
       "    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n",
       "    \n",
       "        &lt;script&gt;\n",
       "            L_NO_TOUCH = false;\n",
       "            L_DISABLE_3D = false;\n",
       "        &lt;/script&gt;\n",
       "    \n",
       "    &lt;style&gt;html, body {width: 100%;height: 100%;margin: 0;padding: 0;}&lt;/style&gt;\n",
       "    &lt;style&gt;#map {position:absolute;top:0;bottom:0;right:0;left:0;}&lt;/style&gt;\n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://code.jquery.com/jquery-1.12.4.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css&quot;/&gt;\n",
       "    \n",
       "            &lt;meta name=&quot;viewport&quot; content=&quot;width=device-width,\n",
       "                initial-scale=1.0, maximum-scale=1.0, user-scalable=no&quot; /&gt;\n",
       "            &lt;style&gt;\n",
       "                #map_13da6d1993af7ee5c12602e3cf427f4c {\n",
       "                    position: relative;\n",
       "                    width: 100.0%;\n",
       "                    height: 100.0%;\n",
       "                    left: 0.0%;\n",
       "                    top: 0.0%;\n",
       "                }\n",
       "            &lt;/style&gt;\n",
       "        \n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/gh/ardhi/Leaflet.MousePosition/src/L.Control.MousePosition.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/ardhi/Leaflet.MousePosition/src/L.Control.MousePosition.min.css&quot;/&gt;\n",
       "&lt;/head&gt;\n",
       "&lt;body&gt;    \n",
       "    \n",
       "            &lt;div class=&quot;folium-map&quot; id=&quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot; &gt;&lt;/div&gt;\n",
       "        \n",
       "&lt;/body&gt;\n",
       "&lt;script&gt;    \n",
       "    \n",
       "            var map_13da6d1993af7ee5c12602e3cf427f4c = L.map(\n",
       "                &quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot;,\n",
       "                {\n",
       "                    center: [29.559684888503615, -95.0830971930759],\n",
       "                    crs: L.CRS.EPSG3857,\n",
       "                    zoom: 5,\n",
       "                    zoomControl: true,\n",
       "                    preferCanvas: false,\n",
       "                }\n",
       "            );\n",
       "\n",
       "            \n",
       "\n",
       "        \n",
       "    \n",
       "            var tile_layer_2166104b19966fa2881a7383d00f452b = L.tileLayer(\n",
       "                &quot;https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png&quot;,\n",
       "                {&quot;attribution&quot;: &quot;Data by \\u0026copy; \\u003ca href=\\&quot;http://openstreetmap.org\\&quot;\\u003eOpenStreetMap\\u003c/a\\u003e, under \\u003ca href=\\&quot;http://www.openstreetmap.org/copyright\\&quot;\\u003eODbL\\u003c/a\\u003e.&quot;, &quot;detectRetina&quot;: false, &quot;maxNativeZoom&quot;: 18, &quot;maxZoom&quot;: 18, &quot;minZoom&quot;: 0, &quot;noWrap&quot;: false, &quot;opacity&quot;: 1, &quot;subdomains&quot;: &quot;abc&quot;, &quot;tms&quot;: false}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var circle_6f0a849de353a501e2395175a3f761f0 = L.circle(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_7eb844a9bb43ea76d5241a8570f61c80 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_bbf6ec21ca8f656b44111c240ed74632 = $(`&lt;div id=&quot;html_bbf6ec21ca8f656b44111c240ed74632&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS LC-40&lt;/div&gt;`)[0];\n",
       "            popup_7eb844a9bb43ea76d5241a8570f61c80.setContent(html_bbf6ec21ca8f656b44111c240ed74632);\n",
       "        \n",
       "\n",
       "        circle_6f0a849de353a501e2395175a3f761f0.bindPopup(popup_7eb844a9bb43ea76d5241a8570f61c80)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_6e7980b6a2dbef7b07a78d8e75c21208 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_8582033e4e89ebf084fe0a34280e5a10 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS LC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_6e7980b6a2dbef7b07a78d8e75c21208.setIcon(div_icon_8582033e4e89ebf084fe0a34280e5a10);\n",
       "        \n",
       "    \n",
       "            var circle_9ddbd8d227d09fbd16df7f474f9a4965 = L.circle(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_27b3883f7919a1a90bd34cf2dee3d822 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_7a18c57cd9ce75b6155af7fd2f507f6d = $(`&lt;div id=&quot;html_7a18c57cd9ce75b6155af7fd2f507f6d&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS SLC-40&lt;/div&gt;`)[0];\n",
       "            popup_27b3883f7919a1a90bd34cf2dee3d822.setContent(html_7a18c57cd9ce75b6155af7fd2f507f6d);\n",
       "        \n",
       "\n",
       "        circle_9ddbd8d227d09fbd16df7f474f9a4965.bindPopup(popup_27b3883f7919a1a90bd34cf2dee3d822)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_04ad8130bd8c6de62c039df0fd4311e6 = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_652a00034a9bd07f40f830803c7a8298 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS SLC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_04ad8130bd8c6de62c039df0fd4311e6.setIcon(div_icon_652a00034a9bd07f40f830803c7a8298);\n",
       "        \n",
       "    \n",
       "            var circle_9725ee5d1199c9a997aeb208f10a5ef1 = L.circle(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_b9d362124e2a8b78cb4e789073c010bb = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_c27d42fbb1f22f4d0374ed551a2b1239 = $(`&lt;div id=&quot;html_c27d42fbb1f22f4d0374ed551a2b1239&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;KSC LC-39A&lt;/div&gt;`)[0];\n",
       "            popup_b9d362124e2a8b78cb4e789073c010bb.setContent(html_c27d42fbb1f22f4d0374ed551a2b1239);\n",
       "        \n",
       "\n",
       "        circle_9725ee5d1199c9a997aeb208f10a5ef1.bindPopup(popup_b9d362124e2a8b78cb4e789073c010bb)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_494cb0aea92783a942965a83480e1270 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_c5c35e200cae27c8f4dc2254b5a48635 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eKSC LC-39A\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_494cb0aea92783a942965a83480e1270.setIcon(div_icon_c5c35e200cae27c8f4dc2254b5a48635);\n",
       "        \n",
       "    \n",
       "            var circle_cfcec0e8a1aaa030b605e4e24669f0ef = L.circle(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_fe181cc238567a1b0d791b16717c7900 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_cba202f338d926d70f7c2be069646b43 = $(`&lt;div id=&quot;html_cba202f338d926d70f7c2be069646b43&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;VAFB SLC-4E&lt;/div&gt;`)[0];\n",
       "            popup_fe181cc238567a1b0d791b16717c7900.setContent(html_cba202f338d926d70f7c2be069646b43);\n",
       "        \n",
       "\n",
       "        circle_cfcec0e8a1aaa030b605e4e24669f0ef.bindPopup(popup_fe181cc238567a1b0d791b16717c7900)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_8b1736b25197fb6337072b733684a683 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_a3e0cd94354b2b0f7c5690b8129667fc = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eVAFB SLC-4E\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_8b1736b25197fb6337072b733684a683.setIcon(div_icon_a3e0cd94354b2b0f7c5690b8129667fc);\n",
       "        \n",
       "    \n",
       "            var mouse_position_12f39c56fd7cd89bffa34218c48112aa = new L.Control.MousePosition(\n",
       "                {&quot;emptyString&quot;: &quot;NaN&quot;, &quot;lngFirst&quot;: false, &quot;numDigits&quot;: 20, &quot;position&quot;: &quot;topright&quot;, &quot;prefix&quot;: &quot;Lat:&quot;, &quot;separator&quot;: &quot; Long: &quot;}\n",
       "            );\n",
       "            mouse_position_12f39c56fd7cd89bffa34218c48112aa.options[&quot;latFormatter&quot;] =\n",
       "                function(num) {return L.Util.formatNum(num, 5);};;\n",
       "            mouse_position_12f39c56fd7cd89bffa34218c48112aa.options[&quot;lngFormatter&quot;] =\n",
       "                function(num) {return L.Util.formatNum(num, 5);};;\n",
       "            map_13da6d1993af7ee5c12602e3cf427f4c.addControl(mouse_position_12f39c56fd7cd89bffa34218c48112aa);\n",
       "        \n",
       "    \n",
       "            var marker_9a2c5ad8d84fe2b52082080b725533f6 = L.marker(\n",
       "                [28.56385, -80.56807],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_5d5f53366297d6042cf882b5cbd34829 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e      0.86 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_9a2c5ad8d84fe2b52082080b725533f6.setIcon(div_icon_5d5f53366297d6042cf882b5cbd34829);\n",
       "        \n",
       "&lt;/script&gt;\" style=\"position:absolute;width:100%;height:100%;left:0;top:0;border:none !important;\" allowfullscreen webkitallowfullscreen mozallowfullscreen></iframe></div></div>"
      ],
      "text/plain": [
       "<folium.folium.Map at 0x228ad5f7070>"
      ]
     },
     "execution_count": 17,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Create and add a folium.Marker on your selected closest coastline point on the map\n",
    "# Display the distance between coastline point and launch site using the icon property \n",
    "# for example\n",
    "# distance_marker = folium.Marker(\n",
    "#    coordinate,\n",
    "#    icon=DivIcon(\n",
    "#        icon_size=(20,20),\n",
    "#        icon_anchor=(0,0),\n",
    "#        html='<div style=\"font-size: 12; color:#d35400;\"><b>%s</b></div>' % \"{:10.2f} KM\".format(distance),\n",
    "#        )\n",
    "#    )\n",
    "\n",
    "distance_marker = folium.Marker(\n",
    "    coastline_coord,\n",
    "    icon=DivIcon(\n",
    "        icon_size=(20,20),\n",
    "        icon_anchor=(0,0),\n",
    "        html='<div style=\"font-size: 12; color:#d35400;\"><b>%s</b></div>' % \"{:10.2f} KM\".format(distance_coastline),\n",
    "        )\n",
    "    )\n",
    "site_map.add_child(distance_marker)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div style=\"width:100%;\"><div style=\"position:relative;width:100%;height:0;padding-bottom:60%;\"><span style=\"color:#565656\">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc=\"&lt;!DOCTYPE html&gt;\n",
       "&lt;head&gt;    \n",
       "    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n",
       "    \n",
       "        &lt;script&gt;\n",
       "            L_NO_TOUCH = false;\n",
       "            L_DISABLE_3D = false;\n",
       "        &lt;/script&gt;\n",
       "    \n",
       "    &lt;style&gt;html, body {width: 100%;height: 100%;margin: 0;padding: 0;}&lt;/style&gt;\n",
       "    &lt;style&gt;#map {position:absolute;top:0;bottom:0;right:0;left:0;}&lt;/style&gt;\n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://code.jquery.com/jquery-1.12.4.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css&quot;/&gt;\n",
       "    \n",
       "            &lt;meta name=&quot;viewport&quot; content=&quot;width=device-width,\n",
       "                initial-scale=1.0, maximum-scale=1.0, user-scalable=no&quot; /&gt;\n",
       "            &lt;style&gt;\n",
       "                #map_13da6d1993af7ee5c12602e3cf427f4c {\n",
       "                    position: relative;\n",
       "                    width: 100.0%;\n",
       "                    height: 100.0%;\n",
       "                    left: 0.0%;\n",
       "                    top: 0.0%;\n",
       "                }\n",
       "            &lt;/style&gt;\n",
       "        \n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/gh/ardhi/Leaflet.MousePosition/src/L.Control.MousePosition.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/ardhi/Leaflet.MousePosition/src/L.Control.MousePosition.min.css&quot;/&gt;\n",
       "&lt;/head&gt;\n",
       "&lt;body&gt;    \n",
       "    \n",
       "            &lt;div class=&quot;folium-map&quot; id=&quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot; &gt;&lt;/div&gt;\n",
       "        \n",
       "&lt;/body&gt;\n",
       "&lt;script&gt;    \n",
       "    \n",
       "            var map_13da6d1993af7ee5c12602e3cf427f4c = L.map(\n",
       "                &quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot;,\n",
       "                {\n",
       "                    center: [29.559684888503615, -95.0830971930759],\n",
       "                    crs: L.CRS.EPSG3857,\n",
       "                    zoom: 5,\n",
       "                    zoomControl: true,\n",
       "                    preferCanvas: false,\n",
       "                }\n",
       "            );\n",
       "\n",
       "            \n",
       "\n",
       "        \n",
       "    \n",
       "            var tile_layer_2166104b19966fa2881a7383d00f452b = L.tileLayer(\n",
       "                &quot;https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png&quot;,\n",
       "                {&quot;attribution&quot;: &quot;Data by \\u0026copy; \\u003ca href=\\&quot;http://openstreetmap.org\\&quot;\\u003eOpenStreetMap\\u003c/a\\u003e, under \\u003ca href=\\&quot;http://www.openstreetmap.org/copyright\\&quot;\\u003eODbL\\u003c/a\\u003e.&quot;, &quot;detectRetina&quot;: false, &quot;maxNativeZoom&quot;: 18, &quot;maxZoom&quot;: 18, &quot;minZoom&quot;: 0, &quot;noWrap&quot;: false, &quot;opacity&quot;: 1, &quot;subdomains&quot;: &quot;abc&quot;, &quot;tms&quot;: false}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var circle_6f0a849de353a501e2395175a3f761f0 = L.circle(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_7eb844a9bb43ea76d5241a8570f61c80 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_bbf6ec21ca8f656b44111c240ed74632 = $(`&lt;div id=&quot;html_bbf6ec21ca8f656b44111c240ed74632&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS LC-40&lt;/div&gt;`)[0];\n",
       "            popup_7eb844a9bb43ea76d5241a8570f61c80.setContent(html_bbf6ec21ca8f656b44111c240ed74632);\n",
       "        \n",
       "\n",
       "        circle_6f0a849de353a501e2395175a3f761f0.bindPopup(popup_7eb844a9bb43ea76d5241a8570f61c80)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_6e7980b6a2dbef7b07a78d8e75c21208 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_8582033e4e89ebf084fe0a34280e5a10 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS LC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_6e7980b6a2dbef7b07a78d8e75c21208.setIcon(div_icon_8582033e4e89ebf084fe0a34280e5a10);\n",
       "        \n",
       "    \n",
       "            var circle_9ddbd8d227d09fbd16df7f474f9a4965 = L.circle(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_27b3883f7919a1a90bd34cf2dee3d822 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_7a18c57cd9ce75b6155af7fd2f507f6d = $(`&lt;div id=&quot;html_7a18c57cd9ce75b6155af7fd2f507f6d&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS SLC-40&lt;/div&gt;`)[0];\n",
       "            popup_27b3883f7919a1a90bd34cf2dee3d822.setContent(html_7a18c57cd9ce75b6155af7fd2f507f6d);\n",
       "        \n",
       "\n",
       "        circle_9ddbd8d227d09fbd16df7f474f9a4965.bindPopup(popup_27b3883f7919a1a90bd34cf2dee3d822)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_04ad8130bd8c6de62c039df0fd4311e6 = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_652a00034a9bd07f40f830803c7a8298 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS SLC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_04ad8130bd8c6de62c039df0fd4311e6.setIcon(div_icon_652a00034a9bd07f40f830803c7a8298);\n",
       "        \n",
       "    \n",
       "            var circle_9725ee5d1199c9a997aeb208f10a5ef1 = L.circle(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_b9d362124e2a8b78cb4e789073c010bb = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_c27d42fbb1f22f4d0374ed551a2b1239 = $(`&lt;div id=&quot;html_c27d42fbb1f22f4d0374ed551a2b1239&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;KSC LC-39A&lt;/div&gt;`)[0];\n",
       "            popup_b9d362124e2a8b78cb4e789073c010bb.setContent(html_c27d42fbb1f22f4d0374ed551a2b1239);\n",
       "        \n",
       "\n",
       "        circle_9725ee5d1199c9a997aeb208f10a5ef1.bindPopup(popup_b9d362124e2a8b78cb4e789073c010bb)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_494cb0aea92783a942965a83480e1270 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_c5c35e200cae27c8f4dc2254b5a48635 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eKSC LC-39A\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_494cb0aea92783a942965a83480e1270.setIcon(div_icon_c5c35e200cae27c8f4dc2254b5a48635);\n",
       "        \n",
       "    \n",
       "            var circle_cfcec0e8a1aaa030b605e4e24669f0ef = L.circle(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_fe181cc238567a1b0d791b16717c7900 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_cba202f338d926d70f7c2be069646b43 = $(`&lt;div id=&quot;html_cba202f338d926d70f7c2be069646b43&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;VAFB SLC-4E&lt;/div&gt;`)[0];\n",
       "            popup_fe181cc238567a1b0d791b16717c7900.setContent(html_cba202f338d926d70f7c2be069646b43);\n",
       "        \n",
       "\n",
       "        circle_cfcec0e8a1aaa030b605e4e24669f0ef.bindPopup(popup_fe181cc238567a1b0d791b16717c7900)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_8b1736b25197fb6337072b733684a683 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_a3e0cd94354b2b0f7c5690b8129667fc = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eVAFB SLC-4E\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_8b1736b25197fb6337072b733684a683.setIcon(div_icon_a3e0cd94354b2b0f7c5690b8129667fc);\n",
       "        \n",
       "    \n",
       "            var mouse_position_12f39c56fd7cd89bffa34218c48112aa = new L.Control.MousePosition(\n",
       "                {&quot;emptyString&quot;: &quot;NaN&quot;, &quot;lngFirst&quot;: false, &quot;numDigits&quot;: 20, &quot;position&quot;: &quot;topright&quot;, &quot;prefix&quot;: &quot;Lat:&quot;, &quot;separator&quot;: &quot; Long: &quot;}\n",
       "            );\n",
       "            mouse_position_12f39c56fd7cd89bffa34218c48112aa.options[&quot;latFormatter&quot;] =\n",
       "                function(num) {return L.Util.formatNum(num, 5);};;\n",
       "            mouse_position_12f39c56fd7cd89bffa34218c48112aa.options[&quot;lngFormatter&quot;] =\n",
       "                function(num) {return L.Util.formatNum(num, 5);};;\n",
       "            map_13da6d1993af7ee5c12602e3cf427f4c.addControl(mouse_position_12f39c56fd7cd89bffa34218c48112aa);\n",
       "        \n",
       "    \n",
       "            var marker_9a2c5ad8d84fe2b52082080b725533f6 = L.marker(\n",
       "                [28.56385, -80.56807],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_5d5f53366297d6042cf882b5cbd34829 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e      0.86 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_9a2c5ad8d84fe2b52082080b725533f6.setIcon(div_icon_5d5f53366297d6042cf882b5cbd34829);\n",
       "        \n",
       "    \n",
       "            var poly_line_d2b1a177e88a6b5723b9a40187fab14e = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.56385, -80.56807]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var poly_line_cdb37016afab6920098dcab5106e7afa = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.56385, -80.56807]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "&lt;/script&gt;\" style=\"position:absolute;width:100%;height:100%;left:0;top:0;border:none !important;\" allowfullscreen webkitallowfullscreen mozallowfullscreen></iframe></div></div>"
      ],
      "text/plain": [
       "<folium.folium.Map at 0x228ad5f7070>"
      ]
     },
     "execution_count": 19,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate\n",
    "# lines=folium.PolyLine(locations=coordinates, weight=1)\n",
    "lines=folium.PolyLine(locations=[launch_sites_df[['Lat', 'Long']].iloc[1].tolist(), coastline_coord], weight=1)\n",
    "site_map.add_child(lines)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Your updated map with distance line should look like the following screenshot:\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<center>\n",
    "    <img src=\"https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png\" />\n",
    "</center>\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "A railway map symbol may look like this:\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<center>\n",
    "    <img src=\"https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png\" />\n",
    "</center>\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "A highway map symbol may look like this:\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<center>\n",
    "    <img src=\"https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png\" />\n",
    "</center>\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "A city map symbol may look like this:\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<center>\n",
    "    <img src=\"https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png\" />\n",
    "</center>\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create a marker with distance to a closest city, railway, highway, etc.\n",
    "# Draw a line between the marker to the launch site"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "17.31856959045091\n"
     ]
    }
   ],
   "source": [
    "# Closest City to CCAFS SLC 40:\n",
    "coord = [28.40814, -80.59291]\n",
    "distance = calculate_distance(\n",
    "    launch_sites_df.loc[1, 'Lat'], launch_sites_df.loc[1, 'Long'], \n",
    "    coord[0], coord[1])\n",
    "print(distance)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div style=\"width:100%;\"><div style=\"position:relative;width:100%;height:0;padding-bottom:60%;\"><span style=\"color:#565656\">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc=\"&lt;!DOCTYPE html&gt;\n",
       "&lt;head&gt;    \n",
       "    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n",
       "    \n",
       "        &lt;script&gt;\n",
       "            L_NO_TOUCH = false;\n",
       "            L_DISABLE_3D = false;\n",
       "        &lt;/script&gt;\n",
       "    \n",
       "    &lt;style&gt;html, body {width: 100%;height: 100%;margin: 0;padding: 0;}&lt;/style&gt;\n",
       "    &lt;style&gt;#map {position:absolute;top:0;bottom:0;right:0;left:0;}&lt;/style&gt;\n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://code.jquery.com/jquery-1.12.4.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css&quot;/&gt;\n",
       "    \n",
       "            &lt;meta name=&quot;viewport&quot; content=&quot;width=device-width,\n",
       "                initial-scale=1.0, maximum-scale=1.0, user-scalable=no&quot; /&gt;\n",
       "            &lt;style&gt;\n",
       "                #map_13da6d1993af7ee5c12602e3cf427f4c {\n",
       "                    position: relative;\n",
       "                    width: 100.0%;\n",
       "                    height: 100.0%;\n",
       "                    left: 0.0%;\n",
       "                    top: 0.0%;\n",
       "                }\n",
       "            &lt;/style&gt;\n",
       "        \n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/gh/ardhi/Leaflet.MousePosition/src/L.Control.MousePosition.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/ardhi/Leaflet.MousePosition/src/L.Control.MousePosition.min.css&quot;/&gt;\n",
       "&lt;/head&gt;\n",
       "&lt;body&gt;    \n",
       "    \n",
       "            &lt;div class=&quot;folium-map&quot; id=&quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot; &gt;&lt;/div&gt;\n",
       "        \n",
       "&lt;/body&gt;\n",
       "&lt;script&gt;    \n",
       "    \n",
       "            var map_13da6d1993af7ee5c12602e3cf427f4c = L.map(\n",
       "                &quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot;,\n",
       "                {\n",
       "                    center: [29.559684888503615, -95.0830971930759],\n",
       "                    crs: L.CRS.EPSG3857,\n",
       "                    zoom: 5,\n",
       "                    zoomControl: true,\n",
       "                    preferCanvas: false,\n",
       "                }\n",
       "            );\n",
       "\n",
       "            \n",
       "\n",
       "        \n",
       "    \n",
       "            var tile_layer_2166104b19966fa2881a7383d00f452b = L.tileLayer(\n",
       "                &quot;https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png&quot;,\n",
       "                {&quot;attribution&quot;: &quot;Data by \\u0026copy; \\u003ca href=\\&quot;http://openstreetmap.org\\&quot;\\u003eOpenStreetMap\\u003c/a\\u003e, under \\u003ca href=\\&quot;http://www.openstreetmap.org/copyright\\&quot;\\u003eODbL\\u003c/a\\u003e.&quot;, &quot;detectRetina&quot;: false, &quot;maxNativeZoom&quot;: 18, &quot;maxZoom&quot;: 18, &quot;minZoom&quot;: 0, &quot;noWrap&quot;: false, &quot;opacity&quot;: 1, &quot;subdomains&quot;: &quot;abc&quot;, &quot;tms&quot;: false}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var circle_6f0a849de353a501e2395175a3f761f0 = L.circle(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_7eb844a9bb43ea76d5241a8570f61c80 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_bbf6ec21ca8f656b44111c240ed74632 = $(`&lt;div id=&quot;html_bbf6ec21ca8f656b44111c240ed74632&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS LC-40&lt;/div&gt;`)[0];\n",
       "            popup_7eb844a9bb43ea76d5241a8570f61c80.setContent(html_bbf6ec21ca8f656b44111c240ed74632);\n",
       "        \n",
       "\n",
       "        circle_6f0a849de353a501e2395175a3f761f0.bindPopup(popup_7eb844a9bb43ea76d5241a8570f61c80)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_6e7980b6a2dbef7b07a78d8e75c21208 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_8582033e4e89ebf084fe0a34280e5a10 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS LC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_6e7980b6a2dbef7b07a78d8e75c21208.setIcon(div_icon_8582033e4e89ebf084fe0a34280e5a10);\n",
       "        \n",
       "    \n",
       "            var circle_9ddbd8d227d09fbd16df7f474f9a4965 = L.circle(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_27b3883f7919a1a90bd34cf2dee3d822 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_7a18c57cd9ce75b6155af7fd2f507f6d = $(`&lt;div id=&quot;html_7a18c57cd9ce75b6155af7fd2f507f6d&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS SLC-40&lt;/div&gt;`)[0];\n",
       "            popup_27b3883f7919a1a90bd34cf2dee3d822.setContent(html_7a18c57cd9ce75b6155af7fd2f507f6d);\n",
       "        \n",
       "\n",
       "        circle_9ddbd8d227d09fbd16df7f474f9a4965.bindPopup(popup_27b3883f7919a1a90bd34cf2dee3d822)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_04ad8130bd8c6de62c039df0fd4311e6 = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_652a00034a9bd07f40f830803c7a8298 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS SLC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_04ad8130bd8c6de62c039df0fd4311e6.setIcon(div_icon_652a00034a9bd07f40f830803c7a8298);\n",
       "        \n",
       "    \n",
       "            var circle_9725ee5d1199c9a997aeb208f10a5ef1 = L.circle(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_b9d362124e2a8b78cb4e789073c010bb = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_c27d42fbb1f22f4d0374ed551a2b1239 = $(`&lt;div id=&quot;html_c27d42fbb1f22f4d0374ed551a2b1239&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;KSC LC-39A&lt;/div&gt;`)[0];\n",
       "            popup_b9d362124e2a8b78cb4e789073c010bb.setContent(html_c27d42fbb1f22f4d0374ed551a2b1239);\n",
       "        \n",
       "\n",
       "        circle_9725ee5d1199c9a997aeb208f10a5ef1.bindPopup(popup_b9d362124e2a8b78cb4e789073c010bb)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_494cb0aea92783a942965a83480e1270 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_c5c35e200cae27c8f4dc2254b5a48635 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eKSC LC-39A\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_494cb0aea92783a942965a83480e1270.setIcon(div_icon_c5c35e200cae27c8f4dc2254b5a48635);\n",
       "        \n",
       "    \n",
       "            var circle_cfcec0e8a1aaa030b605e4e24669f0ef = L.circle(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_fe181cc238567a1b0d791b16717c7900 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_cba202f338d926d70f7c2be069646b43 = $(`&lt;div id=&quot;html_cba202f338d926d70f7c2be069646b43&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;VAFB SLC-4E&lt;/div&gt;`)[0];\n",
       "            popup_fe181cc238567a1b0d791b16717c7900.setContent(html_cba202f338d926d70f7c2be069646b43);\n",
       "        \n",
       "\n",
       "        circle_cfcec0e8a1aaa030b605e4e24669f0ef.bindPopup(popup_fe181cc238567a1b0d791b16717c7900)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_8b1736b25197fb6337072b733684a683 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_a3e0cd94354b2b0f7c5690b8129667fc = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eVAFB SLC-4E\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_8b1736b25197fb6337072b733684a683.setIcon(div_icon_a3e0cd94354b2b0f7c5690b8129667fc);\n",
       "        \n",
       "    \n",
       "            var mouse_position_12f39c56fd7cd89bffa34218c48112aa = new L.Control.MousePosition(\n",
       "                {&quot;emptyString&quot;: &quot;NaN&quot;, &quot;lngFirst&quot;: false, &quot;numDigits&quot;: 20, &quot;position&quot;: &quot;topright&quot;, &quot;prefix&quot;: &quot;Lat:&quot;, &quot;separator&quot;: &quot; Long: &quot;}\n",
       "            );\n",
       "            mouse_position_12f39c56fd7cd89bffa34218c48112aa.options[&quot;latFormatter&quot;] =\n",
       "                function(num) {return L.Util.formatNum(num, 5);};;\n",
       "            mouse_position_12f39c56fd7cd89bffa34218c48112aa.options[&quot;lngFormatter&quot;] =\n",
       "                function(num) {return L.Util.formatNum(num, 5);};;\n",
       "            map_13da6d1993af7ee5c12602e3cf427f4c.addControl(mouse_position_12f39c56fd7cd89bffa34218c48112aa);\n",
       "        \n",
       "    \n",
       "            var marker_9a2c5ad8d84fe2b52082080b725533f6 = L.marker(\n",
       "                [28.56385, -80.56807],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_5d5f53366297d6042cf882b5cbd34829 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e      0.86 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_9a2c5ad8d84fe2b52082080b725533f6.setIcon(div_icon_5d5f53366297d6042cf882b5cbd34829);\n",
       "        \n",
       "    \n",
       "            var poly_line_d2b1a177e88a6b5723b9a40187fab14e = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.56385, -80.56807]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var poly_line_cdb37016afab6920098dcab5106e7afa = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.56385, -80.56807]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var marker_1d68523b8c297f4aef0e1560a5c8df6a = L.marker(\n",
       "                [28.40814, -80.59291],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_f1721d00e8f6a435d6df6f0cc528499b = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e     17.32 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_1d68523b8c297f4aef0e1560a5c8df6a.setIcon(div_icon_f1721d00e8f6a435d6df6f0cc528499b);\n",
       "        \n",
       "    \n",
       "            var poly_line_80e769f062b7f48d8a84012aaa8d9e2f = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.40814, -80.59291]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "&lt;/script&gt;\" style=\"position:absolute;width:100%;height:100%;left:0;top:0;border:none !important;\" allowfullscreen webkitallowfullscreen mozallowfullscreen></iframe></div></div>"
      ],
      "text/plain": [
       "<folium.folium.Map at 0x228ad5f7070>"
      ]
     },
     "execution_count": 22,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "distance_marker = folium.Marker(\n",
    "    coord,\n",
    "    icon=DivIcon(\n",
    "        icon_size=(20,20),\n",
    "        icon_anchor=(0,0),\n",
    "        html='<div style=\"font-size: 12; color:#d35400;\"><b>%s</b></div>' % \"{:10.2f} KM\".format(distance),\n",
    "        )\n",
    "    )\n",
    "site_map.add_child(distance_marker)\n",
    "lines=folium.PolyLine(locations=[launch_sites_df[['Lat', 'Long']].iloc[1].tolist(), coord], weight=1)\n",
    "site_map.add_child(lines)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "1.2868509133886374\n"
     ]
    }
   ],
   "source": [
    "# Closest Railway to CCAFS SLC 40:\n",
    "coord = [28.57208, -80.58526]\n",
    "distance = calculate_distance(\n",
    "    launch_sites_df.loc[1, 'Lat'], launch_sites_df.loc[1, 'Long'], \n",
    "    coord[0], coord[1])\n",
    "print(distance)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div style=\"width:100%;\"><div style=\"position:relative;width:100%;height:0;padding-bottom:60%;\"><span style=\"color:#565656\">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc=\"&lt;!DOCTYPE html&gt;\n",
       "&lt;head&gt;    \n",
       "    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n",
       "    \n",
       "        &lt;script&gt;\n",
       "            L_NO_TOUCH = false;\n",
       "            L_DISABLE_3D = false;\n",
       "        &lt;/script&gt;\n",
       "    \n",
       "    &lt;style&gt;html, body {width: 100%;height: 100%;margin: 0;padding: 0;}&lt;/style&gt;\n",
       "    &lt;style&gt;#map {position:absolute;top:0;bottom:0;right:0;left:0;}&lt;/style&gt;\n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://code.jquery.com/jquery-1.12.4.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css&quot;/&gt;\n",
       "    \n",
       "            &lt;meta name=&quot;viewport&quot; content=&quot;width=device-width,\n",
       "                initial-scale=1.0, maximum-scale=1.0, user-scalable=no&quot; /&gt;\n",
       "            &lt;style&gt;\n",
       "                #map_13da6d1993af7ee5c12602e3cf427f4c {\n",
       "                    position: relative;\n",
       "                    width: 100.0%;\n",
       "                    height: 100.0%;\n",
       "                    left: 0.0%;\n",
       "                    top: 0.0%;\n",
       "                }\n",
       "            &lt;/style&gt;\n",
       "        \n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/gh/ardhi/Leaflet.MousePosition/src/L.Control.MousePosition.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/ardhi/Leaflet.MousePosition/src/L.Control.MousePosition.min.css&quot;/&gt;\n",
       "&lt;/head&gt;\n",
       "&lt;body&gt;    \n",
       "    \n",
       "            &lt;div class=&quot;folium-map&quot; id=&quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot; &gt;&lt;/div&gt;\n",
       "        \n",
       "&lt;/body&gt;\n",
       "&lt;script&gt;    \n",
       "    \n",
       "            var map_13da6d1993af7ee5c12602e3cf427f4c = L.map(\n",
       "                &quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot;,\n",
       "                {\n",
       "                    center: [29.559684888503615, -95.0830971930759],\n",
       "                    crs: L.CRS.EPSG3857,\n",
       "                    zoom: 5,\n",
       "                    zoomControl: true,\n",
       "                    preferCanvas: false,\n",
       "                }\n",
       "            );\n",
       "\n",
       "            \n",
       "\n",
       "        \n",
       "    \n",
       "            var tile_layer_2166104b19966fa2881a7383d00f452b = L.tileLayer(\n",
       "                &quot;https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png&quot;,\n",
       "                {&quot;attribution&quot;: &quot;Data by \\u0026copy; \\u003ca href=\\&quot;http://openstreetmap.org\\&quot;\\u003eOpenStreetMap\\u003c/a\\u003e, under \\u003ca href=\\&quot;http://www.openstreetmap.org/copyright\\&quot;\\u003eODbL\\u003c/a\\u003e.&quot;, &quot;detectRetina&quot;: false, &quot;maxNativeZoom&quot;: 18, &quot;maxZoom&quot;: 18, &quot;minZoom&quot;: 0, &quot;noWrap&quot;: false, &quot;opacity&quot;: 1, &quot;subdomains&quot;: &quot;abc&quot;, &quot;tms&quot;: false}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var circle_6f0a849de353a501e2395175a3f761f0 = L.circle(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_7eb844a9bb43ea76d5241a8570f61c80 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_bbf6ec21ca8f656b44111c240ed74632 = $(`&lt;div id=&quot;html_bbf6ec21ca8f656b44111c240ed74632&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS LC-40&lt;/div&gt;`)[0];\n",
       "            popup_7eb844a9bb43ea76d5241a8570f61c80.setContent(html_bbf6ec21ca8f656b44111c240ed74632);\n",
       "        \n",
       "\n",
       "        circle_6f0a849de353a501e2395175a3f761f0.bindPopup(popup_7eb844a9bb43ea76d5241a8570f61c80)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_6e7980b6a2dbef7b07a78d8e75c21208 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_8582033e4e89ebf084fe0a34280e5a10 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS LC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_6e7980b6a2dbef7b07a78d8e75c21208.setIcon(div_icon_8582033e4e89ebf084fe0a34280e5a10);\n",
       "        \n",
       "    \n",
       "            var circle_9ddbd8d227d09fbd16df7f474f9a4965 = L.circle(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_27b3883f7919a1a90bd34cf2dee3d822 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_7a18c57cd9ce75b6155af7fd2f507f6d = $(`&lt;div id=&quot;html_7a18c57cd9ce75b6155af7fd2f507f6d&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS SLC-40&lt;/div&gt;`)[0];\n",
       "            popup_27b3883f7919a1a90bd34cf2dee3d822.setContent(html_7a18c57cd9ce75b6155af7fd2f507f6d);\n",
       "        \n",
       "\n",
       "        circle_9ddbd8d227d09fbd16df7f474f9a4965.bindPopup(popup_27b3883f7919a1a90bd34cf2dee3d822)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_04ad8130bd8c6de62c039df0fd4311e6 = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_652a00034a9bd07f40f830803c7a8298 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS SLC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_04ad8130bd8c6de62c039df0fd4311e6.setIcon(div_icon_652a00034a9bd07f40f830803c7a8298);\n",
       "        \n",
       "    \n",
       "            var circle_9725ee5d1199c9a997aeb208f10a5ef1 = L.circle(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_b9d362124e2a8b78cb4e789073c010bb = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_c27d42fbb1f22f4d0374ed551a2b1239 = $(`&lt;div id=&quot;html_c27d42fbb1f22f4d0374ed551a2b1239&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;KSC LC-39A&lt;/div&gt;`)[0];\n",
       "            popup_b9d362124e2a8b78cb4e789073c010bb.setContent(html_c27d42fbb1f22f4d0374ed551a2b1239);\n",
       "        \n",
       "\n",
       "        circle_9725ee5d1199c9a997aeb208f10a5ef1.bindPopup(popup_b9d362124e2a8b78cb4e789073c010bb)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_494cb0aea92783a942965a83480e1270 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_c5c35e200cae27c8f4dc2254b5a48635 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eKSC LC-39A\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_494cb0aea92783a942965a83480e1270.setIcon(div_icon_c5c35e200cae27c8f4dc2254b5a48635);\n",
       "        \n",
       "    \n",
       "            var circle_cfcec0e8a1aaa030b605e4e24669f0ef = L.circle(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_fe181cc238567a1b0d791b16717c7900 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_cba202f338d926d70f7c2be069646b43 = $(`&lt;div id=&quot;html_cba202f338d926d70f7c2be069646b43&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;VAFB SLC-4E&lt;/div&gt;`)[0];\n",
       "            popup_fe181cc238567a1b0d791b16717c7900.setContent(html_cba202f338d926d70f7c2be069646b43);\n",
       "        \n",
       "\n",
       "        circle_cfcec0e8a1aaa030b605e4e24669f0ef.bindPopup(popup_fe181cc238567a1b0d791b16717c7900)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_8b1736b25197fb6337072b733684a683 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_a3e0cd94354b2b0f7c5690b8129667fc = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eVAFB SLC-4E\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_8b1736b25197fb6337072b733684a683.setIcon(div_icon_a3e0cd94354b2b0f7c5690b8129667fc);\n",
       "        \n",
       "    \n",
       "            var mouse_position_12f39c56fd7cd89bffa34218c48112aa = new L.Control.MousePosition(\n",
       "                {&quot;emptyString&quot;: &quot;NaN&quot;, &quot;lngFirst&quot;: false, &quot;numDigits&quot;: 20, &quot;position&quot;: &quot;topright&quot;, &quot;prefix&quot;: &quot;Lat:&quot;, &quot;separator&quot;: &quot; Long: &quot;}\n",
       "            );\n",
       "            mouse_position_12f39c56fd7cd89bffa34218c48112aa.options[&quot;latFormatter&quot;] =\n",
       "                function(num) {return L.Util.formatNum(num, 5);};;\n",
       "            mouse_position_12f39c56fd7cd89bffa34218c48112aa.options[&quot;lngFormatter&quot;] =\n",
       "                function(num) {return L.Util.formatNum(num, 5);};;\n",
       "            map_13da6d1993af7ee5c12602e3cf427f4c.addControl(mouse_position_12f39c56fd7cd89bffa34218c48112aa);\n",
       "        \n",
       "    \n",
       "            var marker_9a2c5ad8d84fe2b52082080b725533f6 = L.marker(\n",
       "                [28.56385, -80.56807],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_5d5f53366297d6042cf882b5cbd34829 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e      0.86 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_9a2c5ad8d84fe2b52082080b725533f6.setIcon(div_icon_5d5f53366297d6042cf882b5cbd34829);\n",
       "        \n",
       "    \n",
       "            var poly_line_d2b1a177e88a6b5723b9a40187fab14e = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.56385, -80.56807]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var poly_line_cdb37016afab6920098dcab5106e7afa = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.56385, -80.56807]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var marker_1d68523b8c297f4aef0e1560a5c8df6a = L.marker(\n",
       "                [28.40814, -80.59291],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_f1721d00e8f6a435d6df6f0cc528499b = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e     17.32 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_1d68523b8c297f4aef0e1560a5c8df6a.setIcon(div_icon_f1721d00e8f6a435d6df6f0cc528499b);\n",
       "        \n",
       "    \n",
       "            var poly_line_80e769f062b7f48d8a84012aaa8d9e2f = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.40814, -80.59291]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var marker_0a602b44e9c792a4afa83fa69a502fdc = L.marker(\n",
       "                [28.57208, -80.58526],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_9102682028581f35ee99f24d096ef027 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e      1.29 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_0a602b44e9c792a4afa83fa69a502fdc.setIcon(div_icon_9102682028581f35ee99f24d096ef027);\n",
       "        \n",
       "    \n",
       "            var poly_line_aac26e43f4083fb6ceb7f76d446ebad4 = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.57208, -80.58526]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "&lt;/script&gt;\" style=\"position:absolute;width:100%;height:100%;left:0;top:0;border:none !important;\" allowfullscreen webkitallowfullscreen mozallowfullscreen></iframe></div></div>"
      ],
      "text/plain": [
       "<folium.folium.Map at 0x228ad5f7070>"
      ]
     },
     "execution_count": 24,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "distance_marker = folium.Marker(\n",
    "    coord,\n",
    "    icon=DivIcon(\n",
    "        icon_size=(20,20),\n",
    "        icon_anchor=(0,0),\n",
    "        html='<div style=\"font-size: 12; color:#d35400;\"><b>%s</b></div>' % \"{:10.2f} KM\".format(distance),\n",
    "        )\n",
    "    )\n",
    "site_map.add_child(distance_marker)\n",
    "lines=folium.PolyLine(locations=[launch_sites_df[['Lat', 'Long']].iloc[1].tolist(), coord], weight=1)\n",
    "site_map.add_child(lines)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 25,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "0.5844209146608064\n"
     ]
    }
   ],
   "source": [
    "# Closest Highway to CCAFS SLC 40:\n",
    "coord = [28.56365, -80.57086]\n",
    "distance = calculate_distance(\n",
    "    launch_sites_df.loc[1, 'Lat'], launch_sites_df.loc[1, 'Long'], \n",
    "    coord[0], coord[1])\n",
    "print(distance)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 26,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div style=\"width:100%;\"><div style=\"position:relative;width:100%;height:0;padding-bottom:60%;\"><span style=\"color:#565656\">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc=\"&lt;!DOCTYPE html&gt;\n",
       "&lt;head&gt;    \n",
       "    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n",
       "    \n",
       "        &lt;script&gt;\n",
       "            L_NO_TOUCH = false;\n",
       "            L_DISABLE_3D = false;\n",
       "        &lt;/script&gt;\n",
       "    \n",
       "    &lt;style&gt;html, body {width: 100%;height: 100%;margin: 0;padding: 0;}&lt;/style&gt;\n",
       "    &lt;style&gt;#map {position:absolute;top:0;bottom:0;right:0;left:0;}&lt;/style&gt;\n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://code.jquery.com/jquery-1.12.4.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css&quot;/&gt;\n",
       "    \n",
       "            &lt;meta name=&quot;viewport&quot; content=&quot;width=device-width,\n",
       "                initial-scale=1.0, maximum-scale=1.0, user-scalable=no&quot; /&gt;\n",
       "            &lt;style&gt;\n",
       "                #map_13da6d1993af7ee5c12602e3cf427f4c {\n",
       "                    position: relative;\n",
       "                    width: 100.0%;\n",
       "                    height: 100.0%;\n",
       "                    left: 0.0%;\n",
       "                    top: 0.0%;\n",
       "                }\n",
       "            &lt;/style&gt;\n",
       "        \n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/gh/ardhi/Leaflet.MousePosition/src/L.Control.MousePosition.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/ardhi/Leaflet.MousePosition/src/L.Control.MousePosition.min.css&quot;/&gt;\n",
       "&lt;/head&gt;\n",
       "&lt;body&gt;    \n",
       "    \n",
       "            &lt;div class=&quot;folium-map&quot; id=&quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot; &gt;&lt;/div&gt;\n",
       "        \n",
       "&lt;/body&gt;\n",
       "&lt;script&gt;    \n",
       "    \n",
       "            var map_13da6d1993af7ee5c12602e3cf427f4c = L.map(\n",
       "                &quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot;,\n",
       "                {\n",
       "                    center: [29.559684888503615, -95.0830971930759],\n",
       "                    crs: L.CRS.EPSG3857,\n",
       "                    zoom: 5,\n",
       "                    zoomControl: true,\n",
       "                    preferCanvas: false,\n",
       "                }\n",
       "            );\n",
       "\n",
       "            \n",
       "\n",
       "        \n",
       "    \n",
       "            var tile_layer_2166104b19966fa2881a7383d00f452b = L.tileLayer(\n",
       "                &quot;https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png&quot;,\n",
       "                {&quot;attribution&quot;: &quot;Data by \\u0026copy; \\u003ca href=\\&quot;http://openstreetmap.org\\&quot;\\u003eOpenStreetMap\\u003c/a\\u003e, under \\u003ca href=\\&quot;http://www.openstreetmap.org/copyright\\&quot;\\u003eODbL\\u003c/a\\u003e.&quot;, &quot;detectRetina&quot;: false, &quot;maxNativeZoom&quot;: 18, &quot;maxZoom&quot;: 18, &quot;minZoom&quot;: 0, &quot;noWrap&quot;: false, &quot;opacity&quot;: 1, &quot;subdomains&quot;: &quot;abc&quot;, &quot;tms&quot;: false}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var circle_6f0a849de353a501e2395175a3f761f0 = L.circle(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_7eb844a9bb43ea76d5241a8570f61c80 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_bbf6ec21ca8f656b44111c240ed74632 = $(`&lt;div id=&quot;html_bbf6ec21ca8f656b44111c240ed74632&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS LC-40&lt;/div&gt;`)[0];\n",
       "            popup_7eb844a9bb43ea76d5241a8570f61c80.setContent(html_bbf6ec21ca8f656b44111c240ed74632);\n",
       "        \n",
       "\n",
       "        circle_6f0a849de353a501e2395175a3f761f0.bindPopup(popup_7eb844a9bb43ea76d5241a8570f61c80)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_6e7980b6a2dbef7b07a78d8e75c21208 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_8582033e4e89ebf084fe0a34280e5a10 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS LC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_6e7980b6a2dbef7b07a78d8e75c21208.setIcon(div_icon_8582033e4e89ebf084fe0a34280e5a10);\n",
       "        \n",
       "    \n",
       "            var circle_9ddbd8d227d09fbd16df7f474f9a4965 = L.circle(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_27b3883f7919a1a90bd34cf2dee3d822 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_7a18c57cd9ce75b6155af7fd2f507f6d = $(`&lt;div id=&quot;html_7a18c57cd9ce75b6155af7fd2f507f6d&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS SLC-40&lt;/div&gt;`)[0];\n",
       "            popup_27b3883f7919a1a90bd34cf2dee3d822.setContent(html_7a18c57cd9ce75b6155af7fd2f507f6d);\n",
       "        \n",
       "\n",
       "        circle_9ddbd8d227d09fbd16df7f474f9a4965.bindPopup(popup_27b3883f7919a1a90bd34cf2dee3d822)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_04ad8130bd8c6de62c039df0fd4311e6 = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_652a00034a9bd07f40f830803c7a8298 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS SLC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_04ad8130bd8c6de62c039df0fd4311e6.setIcon(div_icon_652a00034a9bd07f40f830803c7a8298);\n",
       "        \n",
       "    \n",
       "            var circle_9725ee5d1199c9a997aeb208f10a5ef1 = L.circle(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_b9d362124e2a8b78cb4e789073c010bb = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_c27d42fbb1f22f4d0374ed551a2b1239 = $(`&lt;div id=&quot;html_c27d42fbb1f22f4d0374ed551a2b1239&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;KSC LC-39A&lt;/div&gt;`)[0];\n",
       "            popup_b9d362124e2a8b78cb4e789073c010bb.setContent(html_c27d42fbb1f22f4d0374ed551a2b1239);\n",
       "        \n",
       "\n",
       "        circle_9725ee5d1199c9a997aeb208f10a5ef1.bindPopup(popup_b9d362124e2a8b78cb4e789073c010bb)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_494cb0aea92783a942965a83480e1270 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_c5c35e200cae27c8f4dc2254b5a48635 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eKSC LC-39A\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_494cb0aea92783a942965a83480e1270.setIcon(div_icon_c5c35e200cae27c8f4dc2254b5a48635);\n",
       "        \n",
       "    \n",
       "            var circle_cfcec0e8a1aaa030b605e4e24669f0ef = L.circle(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_fe181cc238567a1b0d791b16717c7900 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_cba202f338d926d70f7c2be069646b43 = $(`&lt;div id=&quot;html_cba202f338d926d70f7c2be069646b43&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;VAFB SLC-4E&lt;/div&gt;`)[0];\n",
       "            popup_fe181cc238567a1b0d791b16717c7900.setContent(html_cba202f338d926d70f7c2be069646b43);\n",
       "        \n",
       "\n",
       "        circle_cfcec0e8a1aaa030b605e4e24669f0ef.bindPopup(popup_fe181cc238567a1b0d791b16717c7900)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_8b1736b25197fb6337072b733684a683 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_a3e0cd94354b2b0f7c5690b8129667fc = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eVAFB SLC-4E\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_8b1736b25197fb6337072b733684a683.setIcon(div_icon_a3e0cd94354b2b0f7c5690b8129667fc);\n",
       "        \n",
       "    \n",
       "            var mouse_position_12f39c56fd7cd89bffa34218c48112aa = new L.Control.MousePosition(\n",
       "                {&quot;emptyString&quot;: &quot;NaN&quot;, &quot;lngFirst&quot;: false, &quot;numDigits&quot;: 20, &quot;position&quot;: &quot;topright&quot;, &quot;prefix&quot;: &quot;Lat:&quot;, &quot;separator&quot;: &quot; Long: &quot;}\n",
       "            );\n",
       "            mouse_position_12f39c56fd7cd89bffa34218c48112aa.options[&quot;latFormatter&quot;] =\n",
       "                function(num) {return L.Util.formatNum(num, 5);};;\n",
       "            mouse_position_12f39c56fd7cd89bffa34218c48112aa.options[&quot;lngFormatter&quot;] =\n",
       "                function(num) {return L.Util.formatNum(num, 5);};;\n",
       "            map_13da6d1993af7ee5c12602e3cf427f4c.addControl(mouse_position_12f39c56fd7cd89bffa34218c48112aa);\n",
       "        \n",
       "    \n",
       "            var marker_9a2c5ad8d84fe2b52082080b725533f6 = L.marker(\n",
       "                [28.56385, -80.56807],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_5d5f53366297d6042cf882b5cbd34829 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e      0.86 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_9a2c5ad8d84fe2b52082080b725533f6.setIcon(div_icon_5d5f53366297d6042cf882b5cbd34829);\n",
       "        \n",
       "    \n",
       "            var poly_line_d2b1a177e88a6b5723b9a40187fab14e = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.56385, -80.56807]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var poly_line_cdb37016afab6920098dcab5106e7afa = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.56385, -80.56807]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var marker_1d68523b8c297f4aef0e1560a5c8df6a = L.marker(\n",
       "                [28.40814, -80.59291],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_f1721d00e8f6a435d6df6f0cc528499b = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e     17.32 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_1d68523b8c297f4aef0e1560a5c8df6a.setIcon(div_icon_f1721d00e8f6a435d6df6f0cc528499b);\n",
       "        \n",
       "    \n",
       "            var poly_line_80e769f062b7f48d8a84012aaa8d9e2f = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.40814, -80.59291]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var marker_0a602b44e9c792a4afa83fa69a502fdc = L.marker(\n",
       "                [28.57208, -80.58526],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_9102682028581f35ee99f24d096ef027 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e      1.29 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_0a602b44e9c792a4afa83fa69a502fdc.setIcon(div_icon_9102682028581f35ee99f24d096ef027);\n",
       "        \n",
       "    \n",
       "            var poly_line_aac26e43f4083fb6ceb7f76d446ebad4 = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.57208, -80.58526]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var marker_748e9c958b5d3b6147b500a0c48c6e1b = L.marker(\n",
       "                [28.56365, -80.57086],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_728cd9198c57eedf1f6b2902a1425b71 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e      0.58 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_748e9c958b5d3b6147b500a0c48c6e1b.setIcon(div_icon_728cd9198c57eedf1f6b2902a1425b71);\n",
       "        \n",
       "    \n",
       "            var poly_line_5a739161d3d4f02018e26249af3a31ef = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.56365, -80.57086]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "&lt;/script&gt;\" style=\"position:absolute;width:100%;height:100%;left:0;top:0;border:none !important;\" allowfullscreen webkitallowfullscreen mozallowfullscreen></iframe></div></div>"
      ],
      "text/plain": [
       "<folium.folium.Map at 0x228ad5f7070>"
      ]
     },
     "execution_count": 26,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "distance_marker = folium.Marker(\n",
    "    coord,\n",
    "    icon=DivIcon(\n",
    "        icon_size=(20,20),\n",
    "        icon_anchor=(0,0),\n",
    "        html='<div style=\"font-size: 12; color:#d35400;\"><b>%s</b></div>' % \"{:10.2f} KM\".format(distance),\n",
    "        )\n",
    "    )\n",
    "site_map.add_child(distance_marker)\n",
    "lines=folium.PolyLine(locations=[launch_sites_df[['Lat', 'Long']].iloc[1].tolist(), coord], weight=1)\n",
    "site_map.add_child(lines)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 29,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "11.585245953051794\n"
     ]
    }
   ],
   "source": [
    "# Closest City to VAFB SLC-4E:\n",
    "coord = [34.63298, -120.48416]\n",
    "distance = calculate_distance(\n",
    "    launch_sites_df.loc[3, 'Lat'], launch_sites_df.loc[3, 'Long'], \n",
    "    coord[0], coord[1])\n",
    "print(distance)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 30,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div style=\"width:100%;\"><div style=\"position:relative;width:100%;height:0;padding-bottom:60%;\"><span style=\"color:#565656\">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc=\"&lt;!DOCTYPE html&gt;\n",
       "&lt;head&gt;    \n",
       "    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n",
       "    \n",
       "        &lt;script&gt;\n",
       "            L_NO_TOUCH = false;\n",
       "            L_DISABLE_3D = false;\n",
       "        &lt;/script&gt;\n",
       "    \n",
       "    &lt;style&gt;html, body {width: 100%;height: 100%;margin: 0;padding: 0;}&lt;/style&gt;\n",
       "    &lt;style&gt;#map {position:absolute;top:0;bottom:0;right:0;left:0;}&lt;/style&gt;\n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://code.jquery.com/jquery-1.12.4.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;script src=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css&quot;/&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css&quot;/&gt;\n",
       "    \n",
       "            &lt;meta name=&quot;viewport&quot; content=&quot;width=device-width,\n",
       "                initial-scale=1.0, maximum-scale=1.0, user-scalable=no&quot; /&gt;\n",
       "            &lt;style&gt;\n",
       "                #map_13da6d1993af7ee5c12602e3cf427f4c {\n",
       "                    position: relative;\n",
       "                    width: 100.0%;\n",
       "                    height: 100.0%;\n",
       "                    left: 0.0%;\n",
       "                    top: 0.0%;\n",
       "                }\n",
       "            &lt;/style&gt;\n",
       "        \n",
       "    &lt;script src=&quot;https://cdn.jsdelivr.net/gh/ardhi/Leaflet.MousePosition/src/L.Control.MousePosition.min.js&quot;&gt;&lt;/script&gt;\n",
       "    &lt;link rel=&quot;stylesheet&quot; href=&quot;https://cdn.jsdelivr.net/gh/ardhi/Leaflet.MousePosition/src/L.Control.MousePosition.min.css&quot;/&gt;\n",
       "&lt;/head&gt;\n",
       "&lt;body&gt;    \n",
       "    \n",
       "            &lt;div class=&quot;folium-map&quot; id=&quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot; &gt;&lt;/div&gt;\n",
       "        \n",
       "&lt;/body&gt;\n",
       "&lt;script&gt;    \n",
       "    \n",
       "            var map_13da6d1993af7ee5c12602e3cf427f4c = L.map(\n",
       "                &quot;map_13da6d1993af7ee5c12602e3cf427f4c&quot;,\n",
       "                {\n",
       "                    center: [29.559684888503615, -95.0830971930759],\n",
       "                    crs: L.CRS.EPSG3857,\n",
       "                    zoom: 5,\n",
       "                    zoomControl: true,\n",
       "                    preferCanvas: false,\n",
       "                }\n",
       "            );\n",
       "\n",
       "            \n",
       "\n",
       "        \n",
       "    \n",
       "            var tile_layer_2166104b19966fa2881a7383d00f452b = L.tileLayer(\n",
       "                &quot;https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png&quot;,\n",
       "                {&quot;attribution&quot;: &quot;Data by \\u0026copy; \\u003ca href=\\&quot;http://openstreetmap.org\\&quot;\\u003eOpenStreetMap\\u003c/a\\u003e, under \\u003ca href=\\&quot;http://www.openstreetmap.org/copyright\\&quot;\\u003eODbL\\u003c/a\\u003e.&quot;, &quot;detectRetina&quot;: false, &quot;maxNativeZoom&quot;: 18, &quot;maxZoom&quot;: 18, &quot;minZoom&quot;: 0, &quot;noWrap&quot;: false, &quot;opacity&quot;: 1, &quot;subdomains&quot;: &quot;abc&quot;, &quot;tms&quot;: false}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var circle_6f0a849de353a501e2395175a3f761f0 = L.circle(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_7eb844a9bb43ea76d5241a8570f61c80 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_bbf6ec21ca8f656b44111c240ed74632 = $(`&lt;div id=&quot;html_bbf6ec21ca8f656b44111c240ed74632&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS LC-40&lt;/div&gt;`)[0];\n",
       "            popup_7eb844a9bb43ea76d5241a8570f61c80.setContent(html_bbf6ec21ca8f656b44111c240ed74632);\n",
       "        \n",
       "\n",
       "        circle_6f0a849de353a501e2395175a3f761f0.bindPopup(popup_7eb844a9bb43ea76d5241a8570f61c80)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_6e7980b6a2dbef7b07a78d8e75c21208 = L.marker(\n",
       "                [28.56230197, -80.57735648],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_8582033e4e89ebf084fe0a34280e5a10 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS LC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_6e7980b6a2dbef7b07a78d8e75c21208.setIcon(div_icon_8582033e4e89ebf084fe0a34280e5a10);\n",
       "        \n",
       "    \n",
       "            var circle_9ddbd8d227d09fbd16df7f474f9a4965 = L.circle(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_27b3883f7919a1a90bd34cf2dee3d822 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_7a18c57cd9ce75b6155af7fd2f507f6d = $(`&lt;div id=&quot;html_7a18c57cd9ce75b6155af7fd2f507f6d&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;CCAFS SLC-40&lt;/div&gt;`)[0];\n",
       "            popup_27b3883f7919a1a90bd34cf2dee3d822.setContent(html_7a18c57cd9ce75b6155af7fd2f507f6d);\n",
       "        \n",
       "\n",
       "        circle_9ddbd8d227d09fbd16df7f474f9a4965.bindPopup(popup_27b3883f7919a1a90bd34cf2dee3d822)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_04ad8130bd8c6de62c039df0fd4311e6 = L.marker(\n",
       "                [28.56319718, -80.57682003],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_652a00034a9bd07f40f830803c7a8298 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eCCAFS SLC-40\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_04ad8130bd8c6de62c039df0fd4311e6.setIcon(div_icon_652a00034a9bd07f40f830803c7a8298);\n",
       "        \n",
       "    \n",
       "            var circle_9725ee5d1199c9a997aeb208f10a5ef1 = L.circle(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_b9d362124e2a8b78cb4e789073c010bb = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_c27d42fbb1f22f4d0374ed551a2b1239 = $(`&lt;div id=&quot;html_c27d42fbb1f22f4d0374ed551a2b1239&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;KSC LC-39A&lt;/div&gt;`)[0];\n",
       "            popup_b9d362124e2a8b78cb4e789073c010bb.setContent(html_c27d42fbb1f22f4d0374ed551a2b1239);\n",
       "        \n",
       "\n",
       "        circle_9725ee5d1199c9a997aeb208f10a5ef1.bindPopup(popup_b9d362124e2a8b78cb4e789073c010bb)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_494cb0aea92783a942965a83480e1270 = L.marker(\n",
       "                [28.57325457, -80.64689529],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_c5c35e200cae27c8f4dc2254b5a48635 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eKSC LC-39A\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_494cb0aea92783a942965a83480e1270.setIcon(div_icon_c5c35e200cae27c8f4dc2254b5a48635);\n",
       "        \n",
       "    \n",
       "            var circle_cfcec0e8a1aaa030b605e4e24669f0ef = L.circle(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#d35400&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: true, &quot;fillColor&quot;: &quot;#d35400&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;opacity&quot;: 1.0, &quot;radius&quot;: 200, &quot;stroke&quot;: true, &quot;weight&quot;: 3}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "        var popup_fe181cc238567a1b0d791b16717c7900 = L.popup({&quot;maxWidth&quot;: &quot;100%&quot;});\n",
       "\n",
       "        \n",
       "            var html_cba202f338d926d70f7c2be069646b43 = $(`&lt;div id=&quot;html_cba202f338d926d70f7c2be069646b43&quot; style=&quot;width: 100.0%; height: 100.0%;&quot;&gt;VAFB SLC-4E&lt;/div&gt;`)[0];\n",
       "            popup_fe181cc238567a1b0d791b16717c7900.setContent(html_cba202f338d926d70f7c2be069646b43);\n",
       "        \n",
       "\n",
       "        circle_cfcec0e8a1aaa030b605e4e24669f0ef.bindPopup(popup_fe181cc238567a1b0d791b16717c7900)\n",
       "        ;\n",
       "\n",
       "        \n",
       "    \n",
       "    \n",
       "            var marker_8b1736b25197fb6337072b733684a683 = L.marker(\n",
       "                [34.63283416, -120.6107455],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_a3e0cd94354b2b0f7c5690b8129667fc = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003eVAFB SLC-4E\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_8b1736b25197fb6337072b733684a683.setIcon(div_icon_a3e0cd94354b2b0f7c5690b8129667fc);\n",
       "        \n",
       "    \n",
       "            var mouse_position_12f39c56fd7cd89bffa34218c48112aa = new L.Control.MousePosition(\n",
       "                {&quot;emptyString&quot;: &quot;NaN&quot;, &quot;lngFirst&quot;: false, &quot;numDigits&quot;: 20, &quot;position&quot;: &quot;topright&quot;, &quot;prefix&quot;: &quot;Lat:&quot;, &quot;separator&quot;: &quot; Long: &quot;}\n",
       "            );\n",
       "            mouse_position_12f39c56fd7cd89bffa34218c48112aa.options[&quot;latFormatter&quot;] =\n",
       "                function(num) {return L.Util.formatNum(num, 5);};;\n",
       "            mouse_position_12f39c56fd7cd89bffa34218c48112aa.options[&quot;lngFormatter&quot;] =\n",
       "                function(num) {return L.Util.formatNum(num, 5);};;\n",
       "            map_13da6d1993af7ee5c12602e3cf427f4c.addControl(mouse_position_12f39c56fd7cd89bffa34218c48112aa);\n",
       "        \n",
       "    \n",
       "            var marker_9a2c5ad8d84fe2b52082080b725533f6 = L.marker(\n",
       "                [28.56385, -80.56807],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_5d5f53366297d6042cf882b5cbd34829 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e      0.86 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_9a2c5ad8d84fe2b52082080b725533f6.setIcon(div_icon_5d5f53366297d6042cf882b5cbd34829);\n",
       "        \n",
       "    \n",
       "            var poly_line_d2b1a177e88a6b5723b9a40187fab14e = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.56385, -80.56807]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var poly_line_cdb37016afab6920098dcab5106e7afa = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.56385, -80.56807]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var marker_1d68523b8c297f4aef0e1560a5c8df6a = L.marker(\n",
       "                [28.40814, -80.59291],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_f1721d00e8f6a435d6df6f0cc528499b = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e     17.32 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_1d68523b8c297f4aef0e1560a5c8df6a.setIcon(div_icon_f1721d00e8f6a435d6df6f0cc528499b);\n",
       "        \n",
       "    \n",
       "            var poly_line_80e769f062b7f48d8a84012aaa8d9e2f = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.40814, -80.59291]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var marker_0a602b44e9c792a4afa83fa69a502fdc = L.marker(\n",
       "                [28.57208, -80.58526],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_9102682028581f35ee99f24d096ef027 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e      1.29 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_0a602b44e9c792a4afa83fa69a502fdc.setIcon(div_icon_9102682028581f35ee99f24d096ef027);\n",
       "        \n",
       "    \n",
       "            var poly_line_aac26e43f4083fb6ceb7f76d446ebad4 = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.57208, -80.58526]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var marker_748e9c958b5d3b6147b500a0c48c6e1b = L.marker(\n",
       "                [28.56365, -80.57086],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_728cd9198c57eedf1f6b2902a1425b71 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e      0.58 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_748e9c958b5d3b6147b500a0c48c6e1b.setIcon(div_icon_728cd9198c57eedf1f6b2902a1425b71);\n",
       "        \n",
       "    \n",
       "            var poly_line_5a739161d3d4f02018e26249af3a31ef = L.polyline(\n",
       "                [[28.56319718, -80.57682003], [28.56365, -80.57086]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var marker_75f28978b19d5a9e6a57b10d62bde23e = L.marker(\n",
       "                [34.63298, -120.48416],\n",
       "                {}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "    \n",
       "            var div_icon_6a06cde2664a0770c97237883965c819 = L.divIcon({&quot;className&quot;: &quot;empty&quot;, &quot;html&quot;: &quot;\\u003cdiv style=\\&quot;font-size: 12; color:#d35400;\\&quot;\\u003e\\u003cb\\u003e     11.59 KM\\u003c/b\\u003e\\u003c/div\\u003e&quot;, &quot;iconAnchor&quot;: [0, 0], &quot;iconSize&quot;: [20, 20]});\n",
       "            marker_75f28978b19d5a9e6a57b10d62bde23e.setIcon(div_icon_6a06cde2664a0770c97237883965c819);\n",
       "        \n",
       "    \n",
       "            var poly_line_817e012fce2a5aea1a97274635bc88de = L.polyline(\n",
       "                [[34.63283416, -120.6107455], [34.63298, -120.48416]],\n",
       "                {&quot;bubblingMouseEvents&quot;: true, &quot;color&quot;: &quot;#3388ff&quot;, &quot;dashArray&quot;: null, &quot;dashOffset&quot;: null, &quot;fill&quot;: false, &quot;fillColor&quot;: &quot;#3388ff&quot;, &quot;fillOpacity&quot;: 0.2, &quot;fillRule&quot;: &quot;evenodd&quot;, &quot;lineCap&quot;: &quot;round&quot;, &quot;lineJoin&quot;: &quot;round&quot;, &quot;noClip&quot;: false, &quot;opacity&quot;: 1.0, &quot;smoothFactor&quot;: 1.0, &quot;stroke&quot;: true, &quot;weight&quot;: 1}\n",
       "            ).addTo(map_13da6d1993af7ee5c12602e3cf427f4c);\n",
       "        \n",
       "&lt;/script&gt;\" style=\"position:absolute;width:100%;height:100%;left:0;top:0;border:none !important;\" allowfullscreen webkitallowfullscreen mozallowfullscreen></iframe></div></div>"
      ],
      "text/plain": [
       "<folium.folium.Map at 0x228ad5f7070>"
      ]
     },
     "execution_count": 30,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "distance_marker = folium.Marker(\n",
    "    coord,\n",
    "    icon=DivIcon(\n",
    "        icon_size=(20,20),\n",
    "        icon_anchor=(0,0),\n",
    "        html='<div style=\"font-size: 12; color:#d35400;\"><b>%s</b></div>' % \"{:10.2f} KM\".format(distance),\n",
    "        )\n",
    "    )\n",
    "site_map.add_child(distance_marker)\n",
    "lines=folium.PolyLine(locations=[launch_sites_df[['Lat', 'Long']].iloc[3].tolist(), coord], weight=1)\n",
    "site_map.add_child(lines)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "After you plot distance lines to the proximities, you can answer the following questions easily:\n",
    "\n",
    "*   Are launch sites in close proximity to railways?\n",
    "*   Are launch sites in close proximity to highways?\n",
    "*   Are launch sites in close proximity to coastline?\n",
    "*   Do launch sites keep certain distance away from cities?\n",
    "\n",
    "Also please try to explain your findings.\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Answer:**\n",
    "Launch sites are in close proximity to coasts (<1km), railways (>=~1km), highways (<=1km), but they seem to keep certain distance away from cities: The sites at Cape Canaveral have a minimum distance to the next city (Cape Canaveral) of ~17km, whereas the Vandenberg Space Force Base has at least ~11km to the next city (Lompoc)."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Next Steps:\n",
    "\n",
    "Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Experiments..."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 58,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Launch Site</th>\n",
       "      <th>Lat</th>\n",
       "      <th>Long</th>\n",
       "      <th>class</th>\n",
       "      <th>marker_color</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>26</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>27</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>28</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>1</td>\n",
       "      <td>green</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>29</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>1</td>\n",
       "      <td>green</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>30</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>1</td>\n",
       "      <td>green</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>31</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>1</td>\n",
       "      <td>green</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>32</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>33</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>34</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>35</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "    Launch Site        Lat        Long  class marker_color\n",
       "26  VAFB SLC-4E  34.632834 -120.610745      0          red\n",
       "27  VAFB SLC-4E  34.632834 -120.610745      0          red\n",
       "28  VAFB SLC-4E  34.632834 -120.610745      1        green\n",
       "29  VAFB SLC-4E  34.632834 -120.610745      1        green\n",
       "30  VAFB SLC-4E  34.632834 -120.610745      1        green\n",
       "31  VAFB SLC-4E  34.632834 -120.610745      1        green\n",
       "32  VAFB SLC-4E  34.632834 -120.610745      0          red\n",
       "33  VAFB SLC-4E  34.632834 -120.610745      0          red\n",
       "34  VAFB SLC-4E  34.632834 -120.610745      0          red\n",
       "35  VAFB SLC-4E  34.632834 -120.610745      0          red"
      ]
     },
     "execution_count": 58,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "entered_site = 'VAFB SLC-4E'\n",
    "spacex_df[spacex_df['Launch Site']==entered_site]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 63,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Launch Site</th>\n",
       "      <th>Lat</th>\n",
       "      <th>Long</th>\n",
       "      <th>class</th>\n",
       "      <th>marker_color</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>26</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>27</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>32</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>33</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>34</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>35</th>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>34.632834</td>\n",
       "      <td>-120.610745</td>\n",
       "      <td>0</td>\n",
       "      <td>red</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "    Launch Site        Lat        Long  class marker_color\n",
       "26  VAFB SLC-4E  34.632834 -120.610745      0          red\n",
       "27  VAFB SLC-4E  34.632834 -120.610745      0          red\n",
       "32  VAFB SLC-4E  34.632834 -120.610745      0          red\n",
       "33  VAFB SLC-4E  34.632834 -120.610745      0          red\n",
       "34  VAFB SLC-4E  34.632834 -120.610745      0          red\n",
       "35  VAFB SLC-4E  34.632834 -120.610745      0          red"
      ]
     },
     "execution_count": 63,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "spacex_df[(spacex_df['Launch Site']==entered_site) & (spacex_df['class']==0)]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 64,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Unnamed: 0</th>\n",
       "      <th>Flight Number</th>\n",
       "      <th>Launch Site</th>\n",
       "      <th>class</th>\n",
       "      <th>Payload Mass (kg)</th>\n",
       "      <th>Booster Version</th>\n",
       "      <th>Booster Version Category</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>0.00</td>\n",
       "      <td>F9 v1.0  B0003</td>\n",
       "      <td>v1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>1</td>\n",
       "      <td>2</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>0.00</td>\n",
       "      <td>F9 v1.0  B0004</td>\n",
       "      <td>v1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>2</td>\n",
       "      <td>3</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>525.00</td>\n",
       "      <td>F9 v1.0  B0005</td>\n",
       "      <td>v1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>3</td>\n",
       "      <td>4</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>500.00</td>\n",
       "      <td>F9 v1.0  B0006</td>\n",
       "      <td>v1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>4</td>\n",
       "      <td>5</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>677.00</td>\n",
       "      <td>F9 v1.0  B0007</td>\n",
       "      <td>v1.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>5</th>\n",
       "      <td>5</td>\n",
       "      <td>7</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>3170.00</td>\n",
       "      <td>F9 v1.1</td>\n",
       "      <td>v1.1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6</th>\n",
       "      <td>6</td>\n",
       "      <td>8</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>3325.00</td>\n",
       "      <td>F9 v1.1</td>\n",
       "      <td>v1.1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>7</th>\n",
       "      <td>7</td>\n",
       "      <td>9</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>2296.00</td>\n",
       "      <td>F9 v1.1</td>\n",
       "      <td>v1.1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>8</th>\n",
       "      <td>8</td>\n",
       "      <td>10</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>1316.00</td>\n",
       "      <td>F9 v1.1</td>\n",
       "      <td>v1.1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>9</th>\n",
       "      <td>9</td>\n",
       "      <td>11</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>4535.00</td>\n",
       "      <td>F9 v1.1</td>\n",
       "      <td>v1.1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>10</th>\n",
       "      <td>10</td>\n",
       "      <td>12</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>4428.00</td>\n",
       "      <td>F9 v1.1 B1011</td>\n",
       "      <td>v1.1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>11</th>\n",
       "      <td>11</td>\n",
       "      <td>13</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>2216.00</td>\n",
       "      <td>F9 v1.1 B1010</td>\n",
       "      <td>v1.1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>12</th>\n",
       "      <td>12</td>\n",
       "      <td>14</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>2395.00</td>\n",
       "      <td>F9 v1.1 B1012</td>\n",
       "      <td>v1.1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>13</th>\n",
       "      <td>13</td>\n",
       "      <td>15</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>570.00</td>\n",
       "      <td>F9 v1.1 B1013</td>\n",
       "      <td>v1.1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>14</th>\n",
       "      <td>14</td>\n",
       "      <td>16</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>4159.00</td>\n",
       "      <td>F9 v1.1 B1014</td>\n",
       "      <td>v1.1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>15</th>\n",
       "      <td>15</td>\n",
       "      <td>17</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>1898.00</td>\n",
       "      <td>F9 v1.1 B1015</td>\n",
       "      <td>v1.1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>16</th>\n",
       "      <td>16</td>\n",
       "      <td>18</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>4707.00</td>\n",
       "      <td>F9 v1.1 B1016</td>\n",
       "      <td>v1.1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>17</th>\n",
       "      <td>17</td>\n",
       "      <td>19</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>1</td>\n",
       "      <td>1952.00</td>\n",
       "      <td>F9 v1.1 B1018</td>\n",
       "      <td>v1.1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>18</th>\n",
       "      <td>18</td>\n",
       "      <td>20</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>1</td>\n",
       "      <td>2034.00</td>\n",
       "      <td>F9 FT B1019</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>19</th>\n",
       "      <td>19</td>\n",
       "      <td>22</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>5271.00</td>\n",
       "      <td>F9 FT B1020</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>20</th>\n",
       "      <td>20</td>\n",
       "      <td>23</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>1</td>\n",
       "      <td>3136.00</td>\n",
       "      <td>F9 FT B1021.1</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>21</th>\n",
       "      <td>21</td>\n",
       "      <td>24</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>1</td>\n",
       "      <td>4696.00</td>\n",
       "      <td>F9 FT B1022</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>22</th>\n",
       "      <td>22</td>\n",
       "      <td>25</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>1</td>\n",
       "      <td>3100.00</td>\n",
       "      <td>F9 FT B1023.1</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>23</th>\n",
       "      <td>23</td>\n",
       "      <td>26</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>3600.00</td>\n",
       "      <td>F9 FT B1024</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24</th>\n",
       "      <td>24</td>\n",
       "      <td>27</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>1</td>\n",
       "      <td>2257.00</td>\n",
       "      <td>F9 FT B1025.1</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>25</th>\n",
       "      <td>25</td>\n",
       "      <td>28</td>\n",
       "      <td>CCAFS LC-40</td>\n",
       "      <td>1</td>\n",
       "      <td>4600.00</td>\n",
       "      <td>F9 FT B1026</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>26</th>\n",
       "      <td>26</td>\n",
       "      <td>6</td>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>0</td>\n",
       "      <td>500.00</td>\n",
       "      <td>F9 v1.1  B1003</td>\n",
       "      <td>v1.1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>27</th>\n",
       "      <td>27</td>\n",
       "      <td>21</td>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>0</td>\n",
       "      <td>553.00</td>\n",
       "      <td>F9 v1.1 B1017</td>\n",
       "      <td>v1.1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>28</th>\n",
       "      <td>28</td>\n",
       "      <td>29</td>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>1</td>\n",
       "      <td>9600.00</td>\n",
       "      <td>F9 FT B1029.1</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>29</th>\n",
       "      <td>29</td>\n",
       "      <td>37</td>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>1</td>\n",
       "      <td>9600.00</td>\n",
       "      <td>F9 FT B1036.1</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>30</th>\n",
       "      <td>30</td>\n",
       "      <td>40</td>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>1</td>\n",
       "      <td>475.00</td>\n",
       "      <td>F9 FT B1038.1</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>31</th>\n",
       "      <td>31</td>\n",
       "      <td>42</td>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>1</td>\n",
       "      <td>9600.00</td>\n",
       "      <td>F9 B4 B1041.1</td>\n",
       "      <td>B4</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>32</th>\n",
       "      <td>32</td>\n",
       "      <td>46</td>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>0</td>\n",
       "      <td>9600.00</td>\n",
       "      <td>F9 FT  B1036.2</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>33</th>\n",
       "      <td>33</td>\n",
       "      <td>49</td>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>0</td>\n",
       "      <td>2150.00</td>\n",
       "      <td>F9 FT  B1038.2</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>34</th>\n",
       "      <td>34</td>\n",
       "      <td>51</td>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>0</td>\n",
       "      <td>9600.00</td>\n",
       "      <td>F9 B4  B1041.2</td>\n",
       "      <td>B4</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>35</th>\n",
       "      <td>35</td>\n",
       "      <td>55</td>\n",
       "      <td>VAFB SLC-4E</td>\n",
       "      <td>0</td>\n",
       "      <td>6460.00</td>\n",
       "      <td>F9 B4  B1043.2</td>\n",
       "      <td>B4</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>36</th>\n",
       "      <td>36</td>\n",
       "      <td>30</td>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>1</td>\n",
       "      <td>2490.00</td>\n",
       "      <td>F9 FT B1031.1</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>37</th>\n",
       "      <td>37</td>\n",
       "      <td>31</td>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>0</td>\n",
       "      <td>5600.00</td>\n",
       "      <td>F9 FT B1030</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>38</th>\n",
       "      <td>38</td>\n",
       "      <td>32</td>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>1</td>\n",
       "      <td>5300.00</td>\n",
       "      <td>F9 FT  B1021.2</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>39</th>\n",
       "      <td>39</td>\n",
       "      <td>33</td>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>1</td>\n",
       "      <td>3696.65</td>\n",
       "      <td>F9 FT B1032.1</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>40</th>\n",
       "      <td>40</td>\n",
       "      <td>34</td>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>0</td>\n",
       "      <td>6070.00</td>\n",
       "      <td>F9 FT B1034</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>41</th>\n",
       "      <td>41</td>\n",
       "      <td>35</td>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>1</td>\n",
       "      <td>2708.00</td>\n",
       "      <td>F9 FT B1035.1</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>42</th>\n",
       "      <td>42</td>\n",
       "      <td>36</td>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>1</td>\n",
       "      <td>3669.00</td>\n",
       "      <td>F9 FT  B1029.2</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>43</th>\n",
       "      <td>43</td>\n",
       "      <td>38</td>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>0</td>\n",
       "      <td>6761.00</td>\n",
       "      <td>F9 FT B1037</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>44</th>\n",
       "      <td>44</td>\n",
       "      <td>39</td>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>1</td>\n",
       "      <td>3310.00</td>\n",
       "      <td>F9 B4 B1039.1</td>\n",
       "      <td>B4</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>45</th>\n",
       "      <td>45</td>\n",
       "      <td>41</td>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>1</td>\n",
       "      <td>4990.00</td>\n",
       "      <td>F9 B4 B1040.1</td>\n",
       "      <td>B4</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>46</th>\n",
       "      <td>46</td>\n",
       "      <td>43</td>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>1</td>\n",
       "      <td>5200.00</td>\n",
       "      <td>F9 FT  B1031.2</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>47</th>\n",
       "      <td>47</td>\n",
       "      <td>44</td>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>1</td>\n",
       "      <td>3500.00</td>\n",
       "      <td>F9 B4 B1042.1</td>\n",
       "      <td>B4</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>48</th>\n",
       "      <td>48</td>\n",
       "      <td>54</td>\n",
       "      <td>KSC LC-39A</td>\n",
       "      <td>1</td>\n",
       "      <td>3600.00</td>\n",
       "      <td>F9 B5  B1046.1</td>\n",
       "      <td>B5</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>49</th>\n",
       "      <td>49</td>\n",
       "      <td>45</td>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>1</td>\n",
       "      <td>2205.00</td>\n",
       "      <td>F9 FT  B1035.2</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>50</th>\n",
       "      <td>50</td>\n",
       "      <td>47</td>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>1</td>\n",
       "      <td>3696.65</td>\n",
       "      <td>F9 B4 B1043.1</td>\n",
       "      <td>B4</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>51</th>\n",
       "      <td>51</td>\n",
       "      <td>48</td>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>4230.00</td>\n",
       "      <td>F9 FT  B1032.2</td>\n",
       "      <td>FT</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>52</th>\n",
       "      <td>52</td>\n",
       "      <td>50</td>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>6092.00</td>\n",
       "      <td>F9 B4 B1044</td>\n",
       "      <td>B4</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>53</th>\n",
       "      <td>53</td>\n",
       "      <td>52</td>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>2647.00</td>\n",
       "      <td>F9 B4  B1039.2</td>\n",
       "      <td>B4</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>54</th>\n",
       "      <td>54</td>\n",
       "      <td>53</td>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>1</td>\n",
       "      <td>362.00</td>\n",
       "      <td>F9 B4 B1045.1</td>\n",
       "      <td>B4</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>55</th>\n",
       "      <td>55</td>\n",
       "      <td>56</td>\n",
       "      <td>CCAFS SLC-40</td>\n",
       "      <td>0</td>\n",
       "      <td>5384.00</td>\n",
       "      <td>F9 B4  B1040.2</td>\n",
       "      <td>B4</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "    Unnamed: 0  Flight Number   Launch Site  class  Payload Mass (kg)  \\\n",
       "0            0              1   CCAFS LC-40      0               0.00   \n",
       "1            1              2   CCAFS LC-40      0               0.00   \n",
       "2            2              3   CCAFS LC-40      0             525.00   \n",
       "3            3              4   CCAFS LC-40      0             500.00   \n",
       "4            4              5   CCAFS LC-40      0             677.00   \n",
       "5            5              7   CCAFS LC-40      0            3170.00   \n",
       "6            6              8   CCAFS LC-40      0            3325.00   \n",
       "7            7              9   CCAFS LC-40      0            2296.00   \n",
       "8            8             10   CCAFS LC-40      0            1316.00   \n",
       "9            9             11   CCAFS LC-40      0            4535.00   \n",
       "10          10             12   CCAFS LC-40      0            4428.00   \n",
       "11          11             13   CCAFS LC-40      0            2216.00   \n",
       "12          12             14   CCAFS LC-40      0            2395.00   \n",
       "13          13             15   CCAFS LC-40      0             570.00   \n",
       "14          14             16   CCAFS LC-40      0            4159.00   \n",
       "15          15             17   CCAFS LC-40      0            1898.00   \n",
       "16          16             18   CCAFS LC-40      0            4707.00   \n",
       "17          17             19   CCAFS LC-40      1            1952.00   \n",
       "18          18             20   CCAFS LC-40      1            2034.00   \n",
       "19          19             22   CCAFS LC-40      0            5271.00   \n",
       "20          20             23   CCAFS LC-40      1            3136.00   \n",
       "21          21             24   CCAFS LC-40      1            4696.00   \n",
       "22          22             25   CCAFS LC-40      1            3100.00   \n",
       "23          23             26   CCAFS LC-40      0            3600.00   \n",
       "24          24             27   CCAFS LC-40      1            2257.00   \n",
       "25          25             28   CCAFS LC-40      1            4600.00   \n",
       "26          26              6   VAFB SLC-4E      0             500.00   \n",
       "27          27             21   VAFB SLC-4E      0             553.00   \n",
       "28          28             29   VAFB SLC-4E      1            9600.00   \n",
       "29          29             37   VAFB SLC-4E      1            9600.00   \n",
       "30          30             40   VAFB SLC-4E      1             475.00   \n",
       "31          31             42   VAFB SLC-4E      1            9600.00   \n",
       "32          32             46   VAFB SLC-4E      0            9600.00   \n",
       "33          33             49   VAFB SLC-4E      0            2150.00   \n",
       "34          34             51   VAFB SLC-4E      0            9600.00   \n",
       "35          35             55   VAFB SLC-4E      0            6460.00   \n",
       "36          36             30    KSC LC-39A      1            2490.00   \n",
       "37          37             31    KSC LC-39A      0            5600.00   \n",
       "38          38             32    KSC LC-39A      1            5300.00   \n",
       "39          39             33    KSC LC-39A      1            3696.65   \n",
       "40          40             34    KSC LC-39A      0            6070.00   \n",
       "41          41             35    KSC LC-39A      1            2708.00   \n",
       "42          42             36    KSC LC-39A      1            3669.00   \n",
       "43          43             38    KSC LC-39A      0            6761.00   \n",
       "44          44             39    KSC LC-39A      1            3310.00   \n",
       "45          45             41    KSC LC-39A      1            4990.00   \n",
       "46          46             43    KSC LC-39A      1            5200.00   \n",
       "47          47             44    KSC LC-39A      1            3500.00   \n",
       "48          48             54    KSC LC-39A      1            3600.00   \n",
       "49          49             45  CCAFS SLC-40      1            2205.00   \n",
       "50          50             47  CCAFS SLC-40      1            3696.65   \n",
       "51          51             48  CCAFS SLC-40      0            4230.00   \n",
       "52          52             50  CCAFS SLC-40      0            6092.00   \n",
       "53          53             52  CCAFS SLC-40      0            2647.00   \n",
       "54          54             53  CCAFS SLC-40      1             362.00   \n",
       "55          55             56  CCAFS SLC-40      0            5384.00   \n",
       "\n",
       "   Booster Version Booster Version Category  \n",
       "0   F9 v1.0  B0003                     v1.0  \n",
       "1   F9 v1.0  B0004                     v1.0  \n",
       "2   F9 v1.0  B0005                     v1.0  \n",
       "3   F9 v1.0  B0006                     v1.0  \n",
       "4   F9 v1.0  B0007                     v1.0  \n",
       "5          F9 v1.1                     v1.1  \n",
       "6          F9 v1.1                     v1.1  \n",
       "7          F9 v1.1                     v1.1  \n",
       "8          F9 v1.1                     v1.1  \n",
       "9          F9 v1.1                     v1.1  \n",
       "10   F9 v1.1 B1011                     v1.1  \n",
       "11   F9 v1.1 B1010                     v1.1  \n",
       "12   F9 v1.1 B1012                     v1.1  \n",
       "13   F9 v1.1 B1013                     v1.1  \n",
       "14   F9 v1.1 B1014                     v1.1  \n",
       "15   F9 v1.1 B1015                     v1.1  \n",
       "16   F9 v1.1 B1016                     v1.1  \n",
       "17   F9 v1.1 B1018                     v1.1  \n",
       "18     F9 FT B1019                       FT  \n",
       "19     F9 FT B1020                       FT  \n",
       "20   F9 FT B1021.1                       FT  \n",
       "21     F9 FT B1022                       FT  \n",
       "22   F9 FT B1023.1                       FT  \n",
       "23     F9 FT B1024                       FT  \n",
       "24   F9 FT B1025.1                       FT  \n",
       "25     F9 FT B1026                       FT  \n",
       "26  F9 v1.1  B1003                     v1.1  \n",
       "27   F9 v1.1 B1017                     v1.1  \n",
       "28   F9 FT B1029.1                       FT  \n",
       "29   F9 FT B1036.1                       FT  \n",
       "30   F9 FT B1038.1                       FT  \n",
       "31   F9 B4 B1041.1                       B4  \n",
       "32  F9 FT  B1036.2                       FT  \n",
       "33  F9 FT  B1038.2                       FT  \n",
       "34  F9 B4  B1041.2                       B4  \n",
       "35  F9 B4  B1043.2                       B4  \n",
       "36   F9 FT B1031.1                       FT  \n",
       "37     F9 FT B1030                       FT  \n",
       "38  F9 FT  B1021.2                       FT  \n",
       "39   F9 FT B1032.1                       FT  \n",
       "40     F9 FT B1034                       FT  \n",
       "41   F9 FT B1035.1                       FT  \n",
       "42  F9 FT  B1029.2                       FT  \n",
       "43     F9 FT B1037                       FT  \n",
       "44   F9 B4 B1039.1                       B4  \n",
       "45   F9 B4 B1040.1                       B4  \n",
       "46  F9 FT  B1031.2                       FT  \n",
       "47   F9 B4 B1042.1                       B4  \n",
       "48  F9 B5  B1046.1                       B5  \n",
       "49  F9 FT  B1035.2                       FT  \n",
       "50   F9 B4 B1043.1                       B4  \n",
       "51  F9 FT  B1032.2                       FT  \n",
       "52     F9 B4 B1044                       B4  \n",
       "53  F9 B4  B1039.2                       B4  \n",
       "54   F9 B4 B1045.1                       B4  \n",
       "55  F9 B4  B1040.2                       B4  "
      ]
     },
     "execution_count": 64,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "spacex_df_2 = pd.read_csv(\"spacex_launch_dash.csv\")\n",
    "spacex_df_2"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 66,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Booster Version Category\n",
       "B4      0.545455\n",
       "B5      1.000000\n",
       "FT      0.666667\n",
       "v1.0    0.000000\n",
       "v1.1    0.066667\n",
       "Name: class, dtype: float64"
      ]
     },
     "execution_count": 66,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "spacex_df_2.groupby('Booster Version Category')['class'].mean()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 67,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Booster Version Category\n",
       "B4      11\n",
       "B5       1\n",
       "FT      24\n",
       "v1.0     5\n",
       "v1.1    15\n",
       "Name: class, dtype: int64"
      ]
     },
     "execution_count": 67,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "spacex_df_2.groupby('Booster Version Category')['class'].count()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 68,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Booster Version Category\n",
       "B4       6\n",
       "B5       1\n",
       "FT      16\n",
       "v1.0     0\n",
       "v1.1     1\n",
       "Name: class, dtype: int64"
      ]
     },
     "execution_count": 68,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "spacex_df_2.groupby('Booster Version Category')['class'].sum()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Authors\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01)\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Other Contributors\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Joseph Santarcangelo\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Change Log\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |\n",
    "| ----------------- | ------- | ---------- | --------------------------- |\n",
    "| 2021-05-26        | 1.0     | Yan        | Created the initial version |\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Copyright © 2021 IBM Corporation. All rights reserved.\n"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.7"
  },
  "papermill": {
   "duration": 142.196942,
   "end_time": "2020-09-19T06:29:13.341578",
   "environment_variables": {},
   "exception": null,
   "input_path": "__notebook__.ipynb",
   "output_path": "__notebook__.ipynb",
   "parameters": {},
   "start_time": "2020-09-19T06:26:51.144636",
   "version": "2.1.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}